In [ ]:
import os
os.listdir('/kaggle/input/')

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix, roc_curve, confusion_matrix, precision_recall_curve, f1_score
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import torch.nn.functional as F
import os
import multiprocessing

In [ ]:
print(f"Total CPU cores (logical): {multiprocessing.cpu_count()}")
print(f"os.cpu_count(): {os.cpu_count()}")

In [ ]:
# Updated paths to match the exact filenames in your sidebar
meningitis_raw = pd.read_csv('full_encoded_meningitis.csv')
nonmeningitis_raw = pd.read_csv('full_encoded_non_meningitis.xls')

In [ ]:
meningitis_raw = pd.read_csv('E:/Learning/NEU/Amal Lab/Meningitis paper/full_encoded_meningitis.csv')
nonmeningitis_raw = pd.read_csv('E:/Learning/NEU/Amal Lab/Meningitis paper/full_encoded_non_meningitis.xls')

<h2 style="color: yellow;">Elimiate Columns that directly indicate meningits</h2>
<p style="color: yellow;">We excluded columns starting with '32' that directly indicate meningitis, since they would entail data leaking</p>

In [ ]:
# Check whether there are meningitis related columns (starting with '32') in Meningitis dataset
print("Checking for columns starting with '32' in meningitis_raw...")
print("="*60)

# Get all column names
all_columns_men = meningitis_raw.columns.tolist()

# Find columns that start with "32"
columns_starting_32_men = [col for col in all_columns_men if str(col).startswith(('320','321','322'))]

print(f"Total columns in dataset: {len(all_columns_men)}")
print(f"Columns starting with '32': {len(columns_starting_32_men)}")
print()

if columns_starting_32_men:
    print("Found columns starting with '32':")
    print("-" * 40)
    
    # Sort the columns for better readability
    columns_starting_32_sorted_men = sorted(columns_starting_32_men)
    
    for i, col in enumerate(columns_starting_32_sorted_men, 1):
        # Check how many patients have this code
        count = meningitis_raw[col].sum() if col in meningitis_raw.columns else 0
        percentage = (count / len(meningitis_raw)) * 100 if len(meningitis_raw) > 0 else 0
        
        # Check positive rate when this code is present
        if count > 0:
            pos_rate = meningitis_raw[meningitis_raw[col] == 1]['CLASSIFIER'].mean()
        else:
            pos_rate = 0
            
        print(f"{i:2d}. {str(col):<8} - Present in {int(count):4d} patients ({percentage:5.2f}%) - Pos rate: {pos_rate:.3f}")
    
    print()
    print("Detailed analysis of '32' codes:")
    print("-" * 50)
    
    # Group by length for better understanding
    length_groups = {}
    for col in columns_starting_32_sorted_men:
        length = len(str(col))
        if length not in length_groups:
            length_groups[length] = []
        length_groups[length].append(col)
    
    for length in sorted(length_groups.keys()):
        print(f"\nLength {length} codes ({len(length_groups[length])} codes):")
        for col in length_groups[length]:
            count = int(meningitis_raw[col].sum())
            percentage = (count / len(meningitis_raw)) * 100
            if count > 0:
                pos_rate = meningitis_raw[meningitis_raw[col] == 1]['CLASSIFIER'].mean()
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - Meningitis rate: {pos_rate:.3f}")
            else:
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - No patients with this code")

In [ ]:
# Check whether there are meningitis related columns (starting with '32') in non-meningitis dataset
print("Checking for columns starting with '32' in nonmeningitis_raw...")
print("="*60)

# Get all column names
all_columns_nonmen = nonmeningitis_raw.columns.tolist()

# Find columns that start with "32"
columns_starting_32_nonmen = [col for col in all_columns_nonmen if str(col).startswith(('320','321','322'))]

print(f"Total columns in dataset: {len(all_columns_nonmen)}")
print(f"Columns starting with '32': {len(columns_starting_32_nonmen)}")
print()

if columns_starting_32_nonmen:
    print("Found columns starting with '32':")
    print("-" * 40)
    
    # Sort the columns for better readability
    columns_starting_32_sorted_nonmen = sorted(columns_starting_32_nonmen)
    
    for i, col in enumerate(columns_starting_32_sorted_nonmen, 1):
        # Check how many patients have this code
        count = nonmeningitis_raw[col].sum() if col in nonmeningitis_raw.columns else 0
        percentage = (count / len(nonmeningitis_raw)) * 100 if len(nonmeningitis_raw) > 0 else 0
        
        # Check positive rate when this code is present
        if count > 0:
            pos_rate = nonmeningitis_raw[nonmeningitis_raw[col] == 1]['CLASSIFIER'].mean()
        else:
            pos_rate = 0
            
        print(f"{i:2d}. {str(col):<8} - Present in {int(count):4d} patients ({percentage:5.2f}%) - Pos rate: {pos_rate:.3f}")
    
    print()
    print("Detailed analysis of '32' codes:")
    print("-" * 50)
    
    # Group by length for better understanding
    length_groups = {}
    for col in columns_starting_32_sorted_nonmen:
        length = len(str(col))
        if length not in length_groups:
            length_groups[length] = []
        length_groups[length].append(col)
    
    for length in sorted(length_groups.keys()):
        print(f"\nLength {length} codes ({len(length_groups[length])} codes):")
        for col in length_groups[length]:
            count = int(nonmeningitis_raw[col].sum())
            percentage = (count / len(nonmeningitis_raw)) * 100
            if count > 0:
                pos_rate = nonmeningitis_raw[nonmeningitis_raw[col] == 1]['CLASSIFIER'].mean()
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - Meningitis rate: {pos_rate:.3f}")
            else:
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - No patients with this code")

In [ ]:
# # Create new dataset without meningitis code columns
meningitis_clean = meningitis_raw.drop(columns=columns_starting_32_men)

In [ ]:
# Check columns that start with "('320','321','322')"
all_columns_clean_men = meningitis_clean.columns.tolist()

columns_starting_32_clean = [col for col in all_columns_clean_men if str(col).startswith(('320','321','322'))]

print(f"Total columns in dataset: {len(all_columns_clean_men)}")
print(f"Columns starting with '('320','321','322')': {len(columns_starting_32_clean)}")
print()

In [ ]:
(nonmeningitis_raw["CLASSIFIER"] == 1).any()

In [ ]:
print(meningitis_clean.shape)
print(nonmeningitis_raw.shape)
print(meningitis_clean.head)
print(nonmeningitis_raw.head)

In [ ]:
# Ramdonly extract 180 samples for training set and 34 samples for testing set in meningitis cohort
men_180 = meningitis_clean.sample(n=180, random_state=42)
men_34 = meningitis_clean.drop(men_180.index)

In [ ]:
print(men_180.head)
print(men_34.head)

In [ ]:
# Ramdonly extract 38955 samples for training set and 7348 samples for testing set in the non-meningitis cohort
non_men_38955 = nonmeningitis_raw.sample(n=38955, random_state=42)
non_men_7348 = nonmeningitis_raw.drop(non_men_38955.index)

In [ ]:
# Extract samples from non_men_38955 that at least 1 of the top-100 most important features has a value of 1
def extract_negatives_with_topk_ones(non_men_38955: pd.DataFrame,
                                     top_100_features,
                                     k: int = 1) -> pd.DataFrame:
    """
    Keep rows where at least k of the top-100 features == 1 (or True).
    - negative_samples: DataFrame with 0/1 (or bool) feature columns
    - top_100_features: iterable of feature names (list/Index)
    - k: threshold count (default 1)
    """
    cols = [c for c in top_100_features if c in non_men_38955.columns]
    if not cols:
        return non_men_38955.iloc[0:0].copy()

    sub = non_men_38955[cols]
    # Count how many of the selected features are 1/True in each row
    hits = (sub.eq(1) | sub.eq(True)).sum(axis=1)
    return non_men_38955.loc[hits >= k].copy()

# Example
nonmen_1plus = extract_negatives_with_topk_ones(non_men_38955, top_100_features, k=1)
print(nonmen_1plus.shape)

In [ ]:
# Ramdonly extract 180 and 8000 samples for training set in the non-meningitis cohort
non_men_180 = non_men_38955.sample(n=180, random_state=42)

# Randonly extract 180*10 = 1800, 180*20 = 3600, 180*30 = 5400 from 38955 non-meningitis samples for model training with Case-control subgroups
non_men_1800 = non_men_38955.sample(n=1800, random_state=42)
non_men_3600 = non_men_38955.sample(n=3600, random_state=42)
non_men_5400 = non_men_38955.sample(n=5400, random_state=42)

In [ ]:
non_men_180_sim = nonmen_1plus.sample(n=180, random_state=42)
non_men_1800_sim = nonmen_1plus.sample(n=1800, random_state=42)
non_men_3600_sim = nonmen_1plus.sample(n=3600, random_state=42)
non_men_5400_sim = nonmen_1plus.sample(n=5400, random_state=42)

In [ ]:
print(non_men_38955.head)
print(non_men_7348.head)
print(non_men_180.head)
print(non_men_1800.head)
print(non_men_3600.head)
print(non_men_5400.head)
print(non_men_180_sim.head)
print(non_men_1800_sim.head)
print(non_men_3600_sim.head)

In [ ]:
# Combine data for training set and testing set
training_combined = pd.concat([men_180, non_men_38955], ignore_index = True)

# Combine data for training set 
# This is useful for testing purposes or small-scale experiments
training_combined_360 = pd.concat([men_180, non_men_180], ignore_index = True)
training_combined_1980 = pd.concat([men_180, non_men_1800], ignore_index = True)
training_combined_3780 = pd.concat([men_180, non_men_3600], ignore_index = True)
training_combined_5580 = pd.concat([men_180, non_men_5400], ignore_index = True)
testing_combined_7382 = pd.concat([men_34, non_men_7348], ignore_index = True)

In [ ]:
# Combine data for training set of simulation
training_combined_360_sim = pd.concat([men_180, non_men_180_sim], ignore_index = True)
training_combined_1980_sim = pd.concat([men_180, non_men_1800_sim], ignore_index = True)
training_combined_3780_sim = pd.concat([men_180, non_men_3600_sim], ignore_index = True)
training_combined_5580_sim = pd.concat([men_180, non_men_5400_sim], ignore_index = True)

In [ ]:
# Drop SUBJECT_ID of training sets
training_ID_Drop = training_combined.drop('SUBJECT_ID', axis = 1)
training_ID_Drop_360 = training_combined_360.drop('SUBJECT_ID', axis = 1)
training_ID_Drop_1980 = training_combined_1980.drop('SUBJECT_ID', axis = 1)
training_ID_Drop_3780 = training_combined_3780.drop('SUBJECT_ID', axis = 1)
training_ID_Drop_5580 = training_combined_5580.drop('SUBJECT_ID', axis = 1)

# Drop SUBJECT_ID of testing sets
testing_ID_Drop_7382 = testing_combined_7382.drop('SUBJECT_ID', axis = 1)

In [ ]:
training_ID_Drop_360_sim = training_combined_360_sim.drop('SUBJECT_ID', axis = 1)
training_ID_Drop_1980_sim = training_combined_1980_sim.drop('SUBJECT_ID', axis = 1)
training_ID_Drop_3780_sim = training_combined_3780_sim.drop('SUBJECT_ID', axis = 1)
training_ID_Drop_5580_sim = training_combined_5580_sim.drop('SUBJECT_ID', axis = 1)

In [ ]:
# Shuffle the training datasets to ensure random distribution
training_shuffled = training_ID_Drop.sample(frac=1, random_state=42).reset_index(drop=True)
training_shuffled_360 = training_ID_Drop_360.sample(frac=1, random_state=42).reset_index(drop=True)
training_shuffled_1980 = training_ID_Drop_1980.sample(frac=1, random_state=42).reset_index(drop=True)
training_shuffled_3780 = training_ID_Drop_3780.sample(frac=1, random_state=42).reset_index(drop=True)
training_shuffled_5580 = training_ID_Drop_5580.sample(frac=1, random_state=42).reset_index(drop=True)

# Shuffle the testing datasets to ensure random distribution
testing_shuffled_7382 = testing_ID_Drop_7382.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
training_shuffled_360_sim = training_ID_Drop_360_sim.sample(frac=1, random_state=42).reset_index(drop=True)
training_shuffled_1980_sim = training_ID_Drop_1980_sim.sample(frac=1, random_state=42).reset_index(drop=True)
training_shuffled_3780_sim = training_ID_Drop_3780_sim.sample(frac=1, random_state=42).reset_index(drop=True)
training_shuffled_5580_sim = training_ID_Drop_5580_sim.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print(training_shuffled.shape)
print(training_shuffled_360.shape)
print(training_shuffled_1980.shape)
print(training_shuffled_3780.shape)
print(training_shuffled_5580.shape)

print(testing_shuffled_7382.shape)

In [ ]:
# Check for NaN values in each dataset
print("NaN values in training data:\n", training_shuffled.isnull().sum())
print("\n" + "NaN values in training data (360 samples):\n", training_shuffled_360.isnull().sum())
print("\n" + "NaN values in training data (1980 samples):\n", training_shuffled_1980.isnull().sum())
print("\n" + "NaN values in training data (3780 samples):\n", training_shuffled_3780.isnull().sum())
print("\n" + "NaN values in training data (5580 samples):\n", training_shuffled_5580.isnull().sum())
print("\n" + "NaN values in testing data with 7382 samples:\n", testing_shuffled_7382.isnull().sum())

In [ ]:
df_training_360_sim = training_shuffled_360_sim.fillna(0)
df_training_1980_sim = training_shuffled_1980_sim.fillna(0)
df_training_3780_sim = training_shuffled_3780_sim.fillna(0)
df_training_5580_sim = training_shuffled_5580_sim.fillna(0)

In [ ]:
df_training_360_sim.shape
df_training_1980_sim.shape
df_training_3780_sim.shape
df_training_5580_sim.shape

In [ ]:
# Replace NaN values with 0 in each dataset
df_training = training_shuffled.fillna(0)
df_training_360 = training_shuffled_360.fillna(0)
df_training_1980 = training_shuffled_1980.fillna(0)
df_training_3780 = training_shuffled_3780.fillna(0)
df_training_5580 = training_shuffled_5580.fillna(0)
df_testing_7382 = testing_shuffled_7382.fillna(0)
# Verify if NaN values are replaced
print("NaN values in df_training:\n", df_training.isnull().sum())
print("\nNaN values in df_testing:\n", df_testing_7382.isnull().sum())
print("\nNaN values in df_training_1980:\n", df_training_1980.isnull().sum())
print("\nNaN values in df_training_3780:\n", df_training_3780.isnull().sum())
print("\nNaN values in df_training_5580:\n", df_training_5580.isnull().sum())


In [ ]:
# Check duplicate column names in training data
duplicate_columns = df_training_360.columns[df_training_360.columns.duplicated()].unique()
duplicate_columns_1980 = df_training_1980.columns[df_training_1980.columns.duplicated()].unique()
duplicate_columns_3780 = df_training_3780.columns[df_training_3780.columns.duplicated()].unique()
duplicate_columns_5580 = df_training_5580.columns[df_training_5580.columns.duplicated()].unique()

print(duplicate_columns)
print(duplicate_columns_1980)
print(duplicate_columns_3780)
print(duplicate_columns_5580)


In [ ]:
#################################################################################
# Feature Importance Section

<h2 style="color: yellow;">Conduct Feature Importance</h2>
<p style="color: yellow;">1. Find out features that are most related to meningitis;<br> 2. A simulated situation in ER that includes 50% non-meningitis cases is needed. </p>

In [ ]:
# Separate features and labels
X = df_training.drop('CLASSIFIER', axis=1)
y = df_training['CLASSIFIER']

print("Class distribution:")
print(y.value_counts())
print(f"Positive class ratio: {y.mean():.4f}")
print()

In [ ]:
# Train Random Forest for feature importance
print("Training Random Forest for feature importance...")
rf_importance = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # Handle class imbalance
)

rf_importance.fit(X, y)

In [ ]:
# Get feature importances
feature_importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_importance.feature_importances_
}).sort_values('importance', ascending=False)

In [ ]:
print(feature_importance_df.head(100))

In [ ]:
print("Top 100 most important features:")
print("-" * 50)
for i, (idx, row) in enumerate(feature_importance_df.head(100).iterrows(), 1):
    print(f"{i:2d}. {row['feature']:8s} : {row['importance']:.6f}")

print("/n" + "Feature importance distribution:")
print(f"Max importance: {feature_importance_df['importance'].max():.6f}")
print(f"Mean importance: {feature_importance_df['importance'].mean():.6f}")
print(f"Min importance: {feature_importance_df['importance'].min():.6f}")
print(f"Features with importance > 0.001: {(feature_importance_df['importance'] > 0.001).sum()}")

In [ ]:
# Select top 100 features
top_100_features = feature_importance_df.head(100)['feature'].tolist()

print("Top 100 most important features selected:")
print(f"Feature importance range: {top_100_features[0]} ({feature_importance_df.iloc[0]['importance']:.6f}) to {top_100_features[99]} ({feature_importance_df.iloc[99]['importance']:.6f})")
print()

In [ ]:
# Create new training dataset with top 50 features + target
df_training_important = df_training[top_100_features + ['CLASSIFIER']].copy()

print("New dataset created: df_training_important")
print(f"New dataset shape: {df_training_important.shape}")
print(f"Features reduced from {X.shape[1]} to {len(top_100_features)}")
print()

In [ ]:
# Verify the selection worked correctly
print("Verification:")
print(f"All features present: {set(top_100_features).issubset(set(df_training_important.columns))}")
print(f"CLASSIFIER column present: {'CLASSIFIER' in df_training_important.columns}")
print(f"No missing values in new dataset: {df_training_important.isnull().sum().sum() == 0}")

In [ ]:
# Visualize feature importance distribution
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(range(1, 101), feature_importance_df.head(100)['importance'])
plt.title('Top 100 Feature Importances')
plt.xlabel('Feature Rank')
plt.ylabel('Importance')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.hist(feature_importance_df['importance'], bins=50, alpha=0.7)
plt.axvline(feature_importance_df.iloc[99]['importance'], color='red', linestyle='--', 
           label=f'100th feature: {feature_importance_df.iloc[99]["importance"]:.6f}')
plt.title('Feature Importance Distribution')
plt.xlabel('Importance')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.bar(range(1, 21), feature_importance_df.head(20)['importance'])
plt.title('Top 20 Feature Importances')
plt.xlabel('Feature Rank')
plt.ylabel('Importance')
plt.xticks(range(1, 21, 2))
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
cumulative_importance = np.cumsum(feature_importance_df['importance'])
plt.plot(range(1, len(cumulative_importance) + 1), cumulative_importance)
plt.axvline(100, color='red', linestyle='--', label='Top 100 features')
plt.axhline(cumulative_importance[99], color='red', linestyle='--', 
           label=f'Cumulative: {cumulative_importance[99]:.3f}')
plt.title('Cumulative Feature Importance')
plt.xlabel('Number of Features')
plt.ylabel('Cumulative Importance')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
df_training_important.head()

<h2 style="color: yellow;">Handle Testing Dataset</h2>
<p style="color: yellow;">The main idea of this section is that the positive samples ("CLASSIFIER" == 1) are extracted and combined with negative samples ("CLASSIFIER" == 1) in order to test the model performance under the condition of balanced and imbalanced testing datasets. We search samples in negatives where each one has at least 2 similar features extracted from top 100 features.</p>

In [ ]:
# Separate positive and negative samples
positive_samples = df_testing_7382[df_testing_7382['CLASSIFIER'] == 1]
negative_samples = df_testing_7382[df_testing_7382['CLASSIFIER'] == 0]

len(positive_samples)

In [ ]:
# Extract samples from negative_samples that at least 2 of the top-100 most important features has a value of 1
def extract_negatives_with_topk_ones(negative_samples: pd.DataFrame,
                                     top_100_features,
                                     k: int = 2) -> pd.DataFrame:
    """
    Keep rows where at least k of the top-100 features == 1 (or True).
    - negative_samples: DataFrame with 0/1 (or bool) feature columns
    - top_100_features: iterable of feature names (list/Index)
    - k: threshold count (default 1)
    """
    cols = [c for c in top_100_features if c in negative_samples.columns]
    if not cols:
        return negative_samples.iloc[0:0].copy()

    sub = negative_samples[cols]
    # Count how many of the selected features are 1/True in each row
    hits = (sub.eq(1) | sub.eq(True)).sum(axis=1)
    return negative_samples.loc[hits >= k].copy()

# Example
selected_2plus = extract_negatives_with_topk_ones(negative_samples, top_100_features, k=2)
print(selected_2plus.shape)

In [ ]:
# Create multiple testing sets
df_testing_68 = pd.concat([positive_samples, negative_samples.sample(n=34, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_testing_374 = pd.concat([positive_samples, negative_samples.sample(n=340, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_testing_714 = pd.concat([positive_samples, negative_samples.sample(n=680, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_testing_1054 = pd.concat([positive_samples, negative_samples.sample(n=1020, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_testing_7382 = pd.concat([positive_samples, negative_samples.sample(n=7348, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

# Simulation testing sets (no df_testing_7382 due to less similation sample size than original size)
df_testing_68_sim = pd.concat([positive_samples, selected_2plus.sample(n=34, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_testing_374_sim = pd.concat([positive_samples, selected_2plus.sample(n=340, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_testing_714_sim = pd.concat([positive_samples, selected_2plus.sample(n=680, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
df_testing_1054_sim = pd.concat([positive_samples, selected_2plus.sample(n=1020, random_state=42)], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
print(df_testing_374_sim.shape)
print(df_testing_714_sim.shape)
print(df_testing_1054_sim.shape)


print(df_testing_68.shape)
print(df_testing_374.shape)
print(df_testing_714.shape)
print(df_testing_1054.shape)
print(df_testing_7382.shape)


In [ ]:
# Check duplicate column names after concatenation
duplicate_columns_testing_68 = df_testing_68.columns[df_testing_68.columns.duplicated()].unique()

print(df_testing_68.shape)
duplicate_columns_testing_68_sim = df_testing_68_sim.columns[df_testing_68_sim.columns.duplicated()].unique()

print(f'Number of duplicate_columns_testing_68: {duplicate_columns_testing_68}')
print(f'Number of duplicate_columns_testing_68_sim: {duplicate_columns_testing_68_sim}')


In [ ]:
#####################################################################################################
# Random Forest Training Section

<h2 style="color: yellow;">Model Training</h2>
<p style="color: yellow;">1. Random Forest, LightGBM and DNN are trained with Cost-sensitive Learning respectively. <br>2. After training, we are able to get based models and "ouf of fold" data that are used on "meta model" for Ensemble Learning.</p>

In [ ]:
def train_RF(df, batch_size=64, n_splits=5, n_estimators=500):

    # Separate features and target variable
    X = df.drop('CLASSIFIER', axis=1).values
    y = df['CLASSIFIER'].values

    # Cross-validation setup
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {'AUC': [], 'Sensitivity': [], 'Specificity': [], 'PPV': [], 'NPV': [], 'F1-score': []}

    fold_number = 1
    fpr_list = []
    tpr_list = []

    # Placeholder for out-of-fold predictions
    oof_preds = np.zeros(X.shape[0])
    
    # Print the training dataset size
    print(f"Training Random Forest with {X.shape[0]} samples:")

    for train_index, test_index in skf.split(X, y):
        df_name = [name for name, val in globals().items() if val is df][0]

        # Split data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Convert data to numpy arrays
        X_train = X_train.astype(np.float32)
        X_test = X_test.astype(np.float32)

        n_neg = np.sum(y_train == 0)
        n_pos = np.sum(y_train == 1)
        # Standard balanced weight
        weight_ratio = n_neg / n_pos
        # More aggressive: multiply by 2-3x
        aggressive_weight_ratio = weight_ratio * 0.8
        class_weight_dict = {0: 1.0, 1: aggressive_weight_ratio}

        # Define and train Random Forest model
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            n_jobs=-1,
            random_state=42,
            class_weight=class_weight_dict  # Keep this
        )
        model.fit(X_train, y_train)

        # Predictions
        y_pred_probs = model.predict_proba(X_test)[:, 1]
        
        # OPTIMAL THRESHOLD SELECTION - Youden's J statistic
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
        youden_j = tpr - fpr
        optimal_idx = np.argmax(youden_j)
        optimal_threshold = thresholds[optimal_idx]
        
        # Alternative: F1-maximizing threshold
        # from sklearn.metrics import f1_score
        # thresholds_f1 = np.arange(0.1, 0.9, 0.01)
        # f1_scores = [f1_score(y_test, (y_pred_probs >= t).astype(int)) for t in thresholds_f1]
        # optimal_threshold = thresholds_f1[np.argmax(f1_scores)]
        
        # Use optimal threshold instead of 0.5
        y_pred = (y_pred_probs >= optimal_threshold).astype(int)
        
        # Print threshold info for debugging
        if fold_number == 1:
            print(f"  Optimal threshold (fold {fold_number}): {optimal_threshold:.4f}")
            print(f"  Probability range: [{y_pred_probs.min():.4f}, {y_pred_probs.max():.4f}]")
            print(f"  Positive predictions with optimal threshold: {y_pred.sum()} / {len(y_pred)}")

        # Save OOF predictions in the correct indices
        oof_preds[test_index] = y_pred_probs
        
        # Calculate metrics
        auc = roc_auc_score(y_test, y_pred_probs)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        ppv = tp / (tp + fp) if (tp + fp) > 0 else np.nan
        npv = tn / (tn + fn) if (tn + fn) > 0 else np.nan
        f1 = 2 * (ppv * sensitivity) / (ppv + sensitivity) if (ppv + sensitivity) > 0 else np.nan

        # Append metrics for the fold
        metrics['AUC'].append(auc)
        metrics['Sensitivity'].append(sensitivity)
        metrics['Specificity'].append(specificity)
        metrics['PPV'].append(ppv)
        metrics['NPV'].append(npv)
        metrics['F1-score'].append(f1)

        # Collect data for plotting ROC curve
        fpr_list.append(fpr)
        tpr_list.append(tpr)

        fold_number += 1

    # Calculate mean and 95% CI for each metric
    def compute_mean_ci(values):
        mean = np.mean(values)
        lower = np.percentile(values, 2.5)
        upper = np.percentile(values, 97.5)
        return mean, (lower, upper)

    for metric, values in metrics.items():
        mean, ci = compute_mean_ci(values)
        print(f"{metric}: {mean:.4f} (95% CI: {ci[0]:.4f} - {ci[1]:.4f})")

    # Plot ROC curve for each fold
    plt.figure(figsize=(6, 6))
    for i in range(n_splits):
        plt.plot(fpr_list[i], tpr_list[i], label=f'Fold {i+1} (AUC = {metrics["AUC"][i]:.4f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"ROC Curve of {df_name} using Random Forest")
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

    return model, oof_preds

In [ ]:
# Use 180 + 1800 samples (in df_training_198) as training set
model_RF_1980, oof_preds_RF_1980 = train_RF(df_training_1980)

In [ ]:
# Use 180 + 3600 samples (in df_training_3780) as training set
model_RF_3780, oof_preds_RF_3780 = train_RF(df_training_3780)

In [ ]:
# Use 180 + 5400 samples (in df_training_5580) as training set
model_RF_5580, oof_preds_RF_5580 = train_RF(df_training_5580)

In [ ]:
# Use 180 + 38955 samples (in df_training) as training set
model_RF, oof_preds_RF = train_RF(df_training)

In [ ]:
model_RF_360, oof_preds_RF_360 = train_RF(df_training_360)

In [ ]:
#########################################################################
# LightGBM Training Section

In [ ]:
def train_LightGBM(df, batch_size=64, n_splits=5, n_estimators=100):
    # Separate features and target variable
    X = df.drop('CLASSIFIER', axis=1).values
    y = df['CLASSIFIER'].values

    # Cross-validation setup
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {'AUC': [], 'Sensitivity': [], 'Specificity': [], 'PPV': [], 'NPV': [], 'F1-score': []}

    fold_number = 1
    fpr_list = []
    tpr_list = []

    oof_preds = np.zeros(X.shape[0])

    print(f"Training LightGBM with {X.shape[0]} samples:")

    for train_index, test_index in skf.split(X, y):
        df_name = [name for name, val in globals().items() if val is df][0]

        # Split data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Convert data to numpy arrays
        X_train = X_train.astype(np.float32)
        X_test = X_test.astype(np.float32)

        # Calculate class weights - conservative approach for better PPV and Specificity
        n_neg = np.sum(y_train == 0)
        n_pos = np.sum(y_train == 1)
        
        if n_pos > 0:
            pos_weight = n_neg / n_pos
            # Further reduced multiplier to improve PPV and Specificity
            # Lower multiplier reduces false positives
            aggressive_pos_weight = pos_weight * 0.75  # Reduced from 0.9
        else:
            aggressive_pos_weight = 1.0

        # Enhanced LightGBM with hyperparameters optimized for PPV and Specificity
        model = lgb.LGBMClassifier(
            n_estimators=500,
            learning_rate=0.01,
            max_depth=8,  # Further reduced depth to reduce overfitting
            num_leaves=40,  # Further reduced leaves
            min_child_samples=60,  # Further increased to reduce false positives
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=aggressive_pos_weight,  # Cost-sensitive learning
            reg_alpha=0.3,  # Further increased regularization
            reg_lambda=0.3,
            random_state=42,
            verbose=-1,
            boost_from_average=False,
            objective='binary',
            metric='binary_logloss',
        )
        model.fit(X_train, y_train)

        # Predictions
        y_pred_probs = model.predict_proba(X_test)[:, 1]
        
        # OPTIMIZE THRESHOLD: Prioritize PPV and Specificity
        # Use precision-recall curve for better PPV optimization
        precision, recall, thresholds_pr = precision_recall_curve(y_test, y_pred_probs)
        
        best_score = -1
        optimal_threshold = 0.5
        best_metrics = {}
        
        # Target: High PPV (>= 0.25) and High Specificity (>= 0.70) with reasonable sensitivity (>= 0.70)
        min_sensitivity = 0.70  # Minimum acceptable sensitivity
        min_ppv = 0.25  # Minimum PPV target
        min_specificity = 0.70  # Minimum Specificity target
        
        # Strategy 1: Find threshold that maximizes composite score of PPV and Specificity
        for i, threshold in enumerate(thresholds_pr):
            y_pred_temp = (y_pred_probs >= threshold).astype(int)
            
            if np.sum(y_pred_temp) > 0 and np.sum(y_test) > 0:
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred_temp).ravel()
                sensitivity_temp = tp / (tp + fn) if (tp + fn) > 0 else 0
                ppv_temp = precision[i]  # Use precision from PR curve
                specificity_temp = tn / (tn + fp) if (tn + fp) > 0 else 0
                
                # Check if minimum sensitivity is met
                if sensitivity_temp >= min_sensitivity:
                    # Composite score: balance PPV and Specificity (both reduce false positives)
                    # Equal weight on PPV and Specificity, with some weight on sensitivity
                    composite_score = 0.5 * ppv_temp + 0.4 * specificity_temp + 0.1 * sensitivity_temp
                    
                    if composite_score > best_score:
                        best_score = composite_score
                        optimal_threshold = threshold
                        best_metrics = {
                            'sensitivity': sensitivity_temp,
                            'ppv': ppv_temp,
                            'specificity': specificity_temp,
                            'composite': composite_score,
                            'f1': 2 * (ppv_temp * sensitivity_temp) / (ppv_temp + sensitivity_temp + 1e-10)
                        }
        
        # Strategy 2: If no threshold meets all requirements, relax constraints
        if best_score == -1:
            print(f"  Warning: No threshold meets all constraints. Relaxing...")
            
            # Try with relaxed PPV and Specificity
            min_ppv_relaxed = 0.20
            min_specificity_relaxed = 0.65
            
            for i, threshold in enumerate(thresholds_pr):
                y_pred_temp = (y_pred_probs >= threshold).astype(int)
                
                if np.sum(y_pred_temp) > 0 and np.sum(y_test) > 0:
                    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_temp).ravel()
                    sensitivity_temp = tp / (tp + fn) if (tp + fn) > 0 else 0
                    ppv_temp = precision[i]
                    specificity_temp = tn / (tn + fp) if (tn + fp) > 0 else 0
                    
                    if (sensitivity_temp >= min_sensitivity and 
                        ppv_temp >= min_ppv_relaxed and 
                        specificity_temp >= min_specificity_relaxed):
                        composite_score = 0.5 * ppv_temp + 0.4 * specificity_temp + 0.1 * sensitivity_temp
                        if composite_score > best_score:
                            best_score = composite_score
                            optimal_threshold = threshold
                            best_metrics = {
                                'sensitivity': sensitivity_temp,
                                'ppv': ppv_temp,
                                'specificity': specificity_temp,
                                'composite': composite_score,
                                'f1': 2 * (ppv_temp * sensitivity_temp) / (ppv_temp + sensitivity_temp + 1e-10)
                            }
        
        # Strategy 3: If still no threshold, use grid search with PPV+Specificity optimization
        if best_score == -1:
            thresholds_grid = np.arange(0.3, 0.95, 0.01)
            
            for threshold in thresholds_grid:
                y_pred_temp = (y_pred_probs >= threshold).astype(int)
                if np.sum(y_pred_temp) > 0:
                    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_temp).ravel()
                    sensitivity_temp = tp / (tp + fn) if (tp + fn) > 0 else 0
                    ppv_temp = tp / (tp + fp) if (tp + fp) > 0 else 0
                    specificity_temp = tn / (tn + fp) if (tn + fp) > 0 else 0
                    
                    if sensitivity_temp >= 0.65:  # Minimum sensitivity
                        # Optimize for PPV and Specificity
                        composite_score = 0.5 * ppv_temp + 0.4 * specificity_temp + 0.1 * sensitivity_temp
                        if composite_score > best_score:
                            best_score = composite_score
                            optimal_threshold = threshold
                            best_metrics = {
                                'sensitivity': sensitivity_temp,
                                'ppv': ppv_temp,
                                'specificity': specificity_temp,
                                'composite': composite_score,
                                'f1': 2 * (ppv_temp * sensitivity_temp) / (ppv_temp + sensitivity_temp + 1e-10)
                            }
        
        # Use optimal threshold
        y_pred = (y_pred_probs >= optimal_threshold).astype(int)
        
        # Debug info for first fold
        if fold_number == 1:
            print(f"  Optimal threshold (fold {fold_number}): {optimal_threshold:.4f}")
            print(f"  Best composite score: {best_metrics.get('composite', 0):.4f}")
            print(f"  Sensitivity: {best_metrics.get('sensitivity', 0):.4f}, PPV: {best_metrics.get('ppv', 0):.4f}")
            print(f"  Specificity: {best_metrics.get('specificity', 0):.4f}, F1: {best_metrics.get('f1', 0):.4f}")
            print(f"  Scale pos weight: {aggressive_pos_weight:.2f}")

        oof_preds[test_index] = y_pred_probs

        # Calculate metrics
        auc = roc_auc_score(y_test, y_pred_probs)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        ppv = tp / (tp + fp) if (tp + fp) > 0 else np.nan
        npv = tn / (tn + fn) if (tn + fn) > 0 else np.nan

        # Calculate F1-score
        f1 = 2 * (ppv * sensitivity) / (ppv + sensitivity) if (ppv + sensitivity) > 0 else np.nan

        # Append metrics for the fold
        metrics['AUC'].append(auc)
        metrics['Sensitivity'].append(sensitivity)
        metrics['Specificity'].append(specificity)
        metrics['PPV'].append(ppv)
        metrics['NPV'].append(npv)
        metrics['F1-score'].append(f1)

        # Collect data for plotting ROC curve
        fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
        fpr_list.append(fpr)
        tpr_list.append(tpr)

        fold_number += 1

    # Calculate mean and 95% CI for each metric
    def compute_mean_ci(values):
        mean = np.mean(values)
        lower = np.percentile(values, 2.5)
        upper = np.percentile(values, 97.5)
        return mean, (lower, upper)

    for metric, values in metrics.items():
        mean, ci = compute_mean_ci(values)
        print(f"{metric}: {mean:.4f} (95% CI: {ci[0]:.4f} - {ci[1]:.4f})")

    # Calculate and display average F1-score
    avg_f1 = np.mean(metrics['F1-score'])
    print(f"F1-Score Across Folds: {avg_f1:.4f}")

    # Plot ROC curve for each fold
    plt.figure(figsize=(6, 6))
    for i in range(n_splits):
        plt.plot(fpr_list[i], tpr_list[i], label=f'Fold {i+1} (AUC = {metrics["AUC"][i]:.4f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"ROC Curve of {df_name} using LightGBM")
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

    return model, oof_preds

In [ ]:
# Use 180 + 1800 samples (in df_training_1980) as training set
model_LGBM_1980, oof_preds_LGBM_1980 = train_LightGBM(df_training_1980)

In [ ]:
# Use 180 + 3600 samples (in df_training_3780) as training set
model_LGBM_3780, oof_preds_LGBM_3780 = train_LightGBM(df_training_3780)

In [ ]:
# Use 180 + 5400 samples (in df_training_5580) as training set
model_LGBM_5580, oof_preds_LGBM_5580 = train_LightGBM(df_training_5580)

In [ ]:
# Use 180 + 38955 samples (in df_training) as training set
model_LGBM, oof_preds_LGBM = train_LightGBM(df_training)

In [ ]:
model_LGBM_360, oof_preds_LGBM_360 = train_LightGBM(df_training_360)

In [ ]:
#######################################################################################

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpu_count = torch.cuda.device_count()
print(f"Number of GPUs available: {gpu_count}")

In [ ]:
# --- 1. Weighted Focal Loss Class ---
class WeightedFocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2.0, weight=None):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.weight = weight 

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt)**self.gamma * ce_loss
        return focal_loss.mean()

# --- 2. Main Training Function ---
def train_dnn(df, epochs=250, batch_size=128, n_splits=5, learning_rate=0.003):
    # Separate features and target variable
    X = df.drop('CLASSIFIER', axis=1).values
    y = df['CLASSIFIER'].values

    # Cross-validation setup
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {'AUC': [], 'Sensitivity': [], 'Specificity': [], 'PPV': [], 'NPV': [], 'F1-score': []}

    fold_number = 1
    fpr_list = []
    tpr_list = []
    oof_preds = np.zeros(X.shape[0])
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gpu_count = torch.cuda.device_count()
    print(f"Training DNN with {X.shape[0]} samples on {gpu_count} GPU(s):")

    for train_index, test_index in skf.split(X, y):
        # Extract df's variable name for plot title
        df_name = [name for name, val in globals().items() if val is df][0]

        # Split data
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Convert data to PyTorch tensors
        X_train = torch.tensor(X_train_fold, dtype=torch.float32).to(device)
        y_train = torch.tensor(y_train_fold, dtype=torch.long).to(device)
        X_test = torch.tensor(X_test_fold, dtype=torch.float32).to(device)
        y_test = torch.tensor(y_test_fold, dtype=torch.long).to(device)

        # Create DataLoaders (Fixed NameError)
        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

        # Define the DNN model
        input_size = X_train.shape[1]
        model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        ).to(device)

        # Wrap for Multi-GPU
        if gpu_count > 1:
            model = nn.DataParallel(model)

        # Cost-Sensitive Weights
        class_counts = np.bincount(y_train_fold)
        cw = torch.tensor([1.0, class_counts[0] / class_counts[1]], dtype=torch.float32).to(device)

        criterion = WeightedFocalLoss(alpha=1, gamma=2.0, weight=cw)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-5)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.8)

        # Training loop
        for epoch in range(epochs):
            model.train()
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            scheduler.step()

        # Evaluation for the fold
        model.eval()
        y_true = []
        y_pred_probs = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                y_pred_probs.extend(probs)
                y_true.extend(labels.cpu().numpy())

        # Dynamic Thresholding for Sensitivity Fix
        fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
        optimal_idx = np.argmax(tpr - fpr)
        best_threshold = thresholds[optimal_idx]
        
        y_pred = (np.array(y_pred_probs) >= best_threshold).astype(int)
        oof_preds[test_index] = y_pred_probs

        # Calculate metrics
        auc = roc_auc_score(y_true, y_pred_probs)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        ppv = tp / (tp + fp) if (tp + fp) > 0 else np.nan
        npv = tn / (tn + fn) if (tn + fn) > 0 else np.nan
        f1 = 2 * (ppv * sensitivity) / (ppv + sensitivity) if (ppv + sensitivity) > 0 else np.nan

        metrics['AUC'].append(auc)
        metrics['Sensitivity'].append(sensitivity)
        metrics['Specificity'].append(specificity)
        metrics['PPV'].append(ppv)
        metrics['NPV'].append(npv)
        metrics['F1-score'].append(f1)

        fpr_list.append(fpr)
        tpr_list.append(tpr)
        fold_number += 1

    # Calculate mean and 95% CI
    def compute_mean_ci(values):
        mean = np.mean(values)
        lower = np.percentile(values, 2.5)
        upper = np.percentile(values, 97.5)
        return mean, (lower, upper)

    for metric, values in metrics.items():
        mean, ci = compute_mean_ci(values)
        print(f"{metric}: {mean:.4f} (95% CI: {ci[0]:.4f} - {ci[1]:.4f})")

    # Plot ROC curve (Original format)
    plt.figure(figsize=(6, 6))
    for i in range(n_splits):
        plt.plot(fpr_list[i], tpr_list[i], label=f'Fold {i+1} (AUC = {metrics["AUC"][i]:.4f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"ROC Curve of {df_name} using DNN")
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

    return model, oof_preds

In [ ]:
# Use 180 + 1800 samples (in df_training_1980) as training set
model_DNN_1980, oof_preds_DNN_1980 = train_dnn(df_training_1980)

In [ ]:
# Use 180 + 3600 samples (in df_training_3780) as training set
model_DNN_3780, oof_preds_DNN_3780 = train_dnn(df_training_3780)

In [ ]:
# Use 180 + 5400 samples (in df_training_5580) as training set
model_DNN_5580, oof_preds_DNN_5580 = train_dnn(df_training_5580)

In [ ]:
# Use 180 + 38955 samples (in df_training) as training set
model_DNN, oof_preds_DNN = train_dnn(df_training)

In [ ]:
model_DNN_360, oof_preds_DNN_360 = train_dnn(df_training_360)

In [ ]:
########################################################################################

In [ ]:
# Conduct ensemble learning with 8180 and 360 samples

In [ ]:
# X = meta-feature matrix (3 columns: RF, LGBM, DNN predictions)
oof_preds_X_360 = np.column_stack([oof_preds_RF_360, oof_preds_LGBM_360, oof_preds_DNN_360])
oof_preds_X_1980 = np.column_stack([oof_preds_RF_1980, oof_preds_LGBM_1980, oof_preds_DNN_1980])
oof_preds_X_3780 = np.column_stack([oof_preds_RF_3780, oof_preds_LGBM_3780, oof_preds_DNN_3780])
oof_preds_X_5580 = np.column_stack([oof_preds_RF_5580, oof_preds_LGBM_5580, oof_preds_DNN_5580])


# y = true labels for training data
y_meta_360 = df_training_360['CLASSIFIER'].values
y_meta_1980 = df_training_1980['CLASSIFIER'].values
y_meta_3780 = df_training_3780['CLASSIFIER'].values
y_meta_5580 = df_training_5580['CLASSIFIER'].values

In [ ]:
# Train the LR meta-learner
meta_model_LR_360 = LogisticRegression(class_weight='balanced', penalty='l2', solver='lbfgs', max_iter=1000)
meta_model_LR_1980 = LogisticRegression(class_weight='balanced', penalty='l2', solver='lbfgs', max_iter=1000)
meta_model_LR_3780 = LogisticRegression(class_weight='balanced', penalty='l2', solver='lbfgs', max_iter=1000)
meta_model_LR_5580 = LogisticRegression(class_weight='balanced', penalty='l2', solver='lbfgs', max_iter=1000)
meta_model_LR = LogisticRegression(class_weight="balanced", penalty="l2", solver="lbfgs", max_iter=1000)

meta_model_LR_360.fit(oof_preds_X_360, y_meta_360)
meta_model_LR_1980.fit(oof_preds_X_1980, y_meta_1980)
meta_model_LR_3780.fit(oof_preds_X_3780, y_meta_3780)
meta_model_LR_5580.fit(oof_preds_X_5580, y_meta_5580)


In [ ]:
# Prepare test features and labels for 68, 374, 714, 1054, 7382 testing sets 
X_test_68 = df_testing_68.drop('CLASSIFIER', axis=1).values
X_test_374 = df_testing_374.drop('CLASSIFIER', axis=1).values
X_test_714 = df_testing_714.drop('CLASSIFIER', axis=1).values
X_test_1054 = df_testing_1054.drop('CLASSIFIER', axis=1).values
X_test_7382 = df_testing_7382.drop('CLASSIFIER', axis=1).values

# Prepare test features and labels for 68, 374, 714, 1054, 7382 testing sets of simulations
X_test_68_sim = df_testing_68_sim.drop('CLASSIFIER', axis=1).values
X_test_374_sim = df_testing_374_sim.drop('CLASSIFIER', axis=1).values
X_test_714_sim = df_testing_714_sim.drop('CLASSIFIER', axis=1).values
X_test_1054_sim = df_testing_1054_sim.drop('CLASSIFIER', axis=1).values


In [ ]:
# Prepare test features and labels for classifiers

# Regular
y_test_68 = df_testing_68['CLASSIFIER'].values
y_test_374 = df_testing_374['CLASSIFIER'].values
y_test_714 = df_testing_714['CLASSIFIER'].values
y_test_1054 = df_testing_1054['CLASSIFIER'].values
y_test_7382 = df_testing_7382['CLASSIFIER'].values

# Simulation
y_test_68_sim = df_testing_68_sim['CLASSIFIER'].values
y_test_374_sim = df_testing_374_sim['CLASSIFIER'].values
y_test_714_sim = df_testing_714_sim['CLASSIFIER'].values
y_test_1054_sim = df_testing_1054_sim['CLASSIFIER'].values


In [ ]:
# Predict probabilities for ramdon forest
# X_test structure: 34 +34, 34 + 340, 34 + 680, 34 + 1020, 34 + 1364, 34 + 7348  
# Regular 
test_preds_RF_360_68 = model_RF_360.predict_proba(X_test_68)[:, 1]
test_preds_RF_1980_68 = model_RF_1980.predict_proba(X_test_68)[:, 1]
test_preds_RF_3780_68 = model_RF_3780.predict_proba(X_test_68)[:, 1]
test_preds_RF_5580_68 = model_RF_5580.predict_proba(X_test_68)[:, 1]

test_preds_RF_360_374 = model_RF_360.predict_proba(X_test_374)[:, 1]
test_preds_RF_1980_374 = model_RF_1980.predict_proba(X_test_374)[:, 1]
test_preds_RF_3780_374 = model_RF_3780.predict_proba(X_test_374)[:, 1]
test_preds_RF_5580_374 = model_RF_5580.predict_proba(X_test_374)[:, 1]

test_preds_RF_360_714 = model_RF_360.predict_proba(X_test_714)[:, 1]
test_preds_RF_1980_714 = model_RF_1980.predict_proba(X_test_714)[:, 1]
test_preds_RF_3780_714 = model_RF_3780.predict_proba(X_test_714)[:, 1]
test_preds_RF_5580_714 = model_RF_5580.predict_proba(X_test_714)[:, 1]

test_preds_RF_360_1054 = model_RF_360.predict_proba(X_test_1054)[:, 1]
test_preds_RF_1980_1054 = model_RF_1980.predict_proba(X_test_1054)[:, 1]
test_preds_RF_3780_1054 = model_RF_3780.predict_proba(X_test_1054)[:, 1]
test_preds_RF_5580_1054 = model_RF_5580.predict_proba(X_test_1054)[:, 1]

test_preds_RF_360_7382 = model_RF_360.predict_proba(X_test_7382)[:, 1]
test_preds_RF_1980_7382 = model_RF_1980.predict_proba(X_test_7382)[:, 1]
test_preds_RF_3780_7382 = model_RF_3780.predict_proba(X_test_7382)[:, 1]
test_preds_RF_5580_7382 = model_RF_5580.predict_proba(X_test_7382)[:, 1]

# Simulation
test_preds_RF_360_68_sim = model_RF_360.predict_proba(X_test_68_sim)[:, 1]
test_preds_RF_1980_68_sim = model_RF_1980.predict_proba(X_test_68_sim)[:, 1]
test_preds_RF_3780_68_sim = model_RF_3780.predict_proba(X_test_68_sim)[:, 1]
test_preds_RF_5580_68_sim = model_RF_5580.predict_proba(X_test_68_sim)[:, 1]

test_preds_RF_360_374_sim = model_RF_360.predict_proba(X_test_374_sim)[:, 1]
test_preds_RF_1980_374_sim = model_RF_1980.predict_proba(X_test_374_sim)[:, 1]
test_preds_RF_3780_374_sim = model_RF_3780.predict_proba(X_test_374_sim)[:, 1]
test_preds_RF_5580_374_sim = model_RF_5580.predict_proba(X_test_374_sim)[:, 1]

test_preds_RF_360_714_sim = model_RF_360.predict_proba(X_test_714_sim)[:, 1]
test_preds_RF_1980_714_sim = model_RF_1980.predict_proba(X_test_714_sim)[:, 1]
test_preds_RF_3780_714_sim = model_RF_3780.predict_proba(X_test_714_sim)[:, 1]
test_preds_RF_5580_714_sim = model_RF_5580.predict_proba(X_test_714_sim)[:, 1]

test_preds_RF_360_1054_sim = model_RF_360.predict_proba(X_test_1054_sim)[:, 1]
test_preds_RF_1980_1054_sim = model_RF_1980.predict_proba(X_test_1054_sim)[:, 1]
test_preds_RF_3780_1054_sim = model_RF_3780.predict_proba(X_test_1054_sim)[:, 1]
test_preds_RF_5580_1054_sim = model_RF_5580.predict_proba(X_test_1054_sim)[:, 1]


In [ ]:
# Predict probabilities for LightGBM
# X_test structure: 34 +34, 34 + 340, 34 + 680, 34 + 1020, 34 + 1364, 34 + 7348  
# Regular 
test_preds_LGBM_360_68 = model_LGBM_360.predict_proba(X_test_68)[:, 1]
test_preds_LGBM_1980_68 = model_LGBM_1980.predict_proba(X_test_68)[:, 1]
test_preds_LGBM_3780_68 = model_LGBM_3780.predict_proba(X_test_68)[:, 1]
test_preds_LGBM_5580_68 = model_LGBM_5580.predict_proba(X_test_68)[:, 1]

test_preds_LGBM_360_374 = model_LGBM_360.predict_proba(X_test_374)[:, 1]
test_preds_LGBM_1980_374 = model_LGBM_1980.predict_proba(X_test_374)[:, 1]
test_preds_LGBM_3780_374 = model_LGBM_3780.predict_proba(X_test_374)[:, 1]
test_preds_LGBM_5580_374 = model_LGBM_5580.predict_proba(X_test_374)[:, 1]

test_preds_LGBM_360_714 = model_LGBM_360.predict_proba(X_test_714)[:, 1]
test_preds_LGBM_1980_714 = model_LGBM_1980.predict_proba(X_test_714)[:, 1]
test_preds_LGBM_3780_714 = model_LGBM_3780.predict_proba(X_test_714)[:, 1]
test_preds_LGBM_5580_714 = model_LGBM_5580.predict_proba(X_test_714)[:, 1]

test_preds_LGBM_360_1054 = model_LGBM_360.predict_proba(X_test_1054)[:, 1]
test_preds_LGBM_1980_1054 = model_LGBM_1980.predict_proba(X_test_1054)[:, 1]
test_preds_LGBM_3780_1054 = model_LGBM_3780.predict_proba(X_test_1054)[:, 1]
test_preds_LGBM_5580_1054 = model_LGBM_5580.predict_proba(X_test_1054)[:, 1]

test_preds_LGBM_360_7382 = model_LGBM_360.predict_proba(X_test_7382)[:, 1]
test_preds_LGBM_1980_7382 = model_LGBM_1980.predict_proba(X_test_7382)[:, 1]
test_preds_LGBM_3780_7382 = model_LGBM_3780.predict_proba(X_test_7382)[:, 1]
test_preds_LGBM_5580_7382 = model_LGBM_5580.predict_proba(X_test_7382)[:, 1]

# Simulation
test_preds_LGBM_360_68_sim = model_LGBM_360.predict_proba(X_test_68_sim)[:, 1]
test_preds_LGBM_1980_68_sim = model_LGBM_1980.predict_proba(X_test_68_sim)[:, 1]
test_preds_LGBM_3780_68_sim = model_LGBM_3780.predict_proba(X_test_68_sim)[:, 1]
test_preds_LGBM_5580_68_sim = model_LGBM_5580.predict_proba(X_test_68_sim)[:, 1]

test_preds_LGBM_360_374_sim = model_LGBM_360.predict_proba(X_test_374_sim)[:, 1]
test_preds_LGBM_1980_374_sim = model_LGBM_1980.predict_proba(X_test_374_sim)[:, 1]
test_preds_LGBM_3780_374_sim = model_LGBM_3780.predict_proba(X_test_374_sim)[:, 1]
test_preds_LGBM_5580_374_sim = model_LGBM_5580.predict_proba(X_test_374_sim)[:, 1]

test_preds_LGBM_360_714_sim = model_LGBM_360.predict_proba(X_test_714_sim)[:, 1]
test_preds_LGBM_1980_714_sim = model_LGBM_1980.predict_proba(X_test_714_sim)[:, 1]
test_preds_LGBM_3780_714_sim = model_LGBM_3780.predict_proba(X_test_714_sim)[:, 1]
test_preds_LGBM_5580_714_sim = model_LGBM_5580.predict_proba(X_test_714_sim)[:, 1]

test_preds_LGBM_360_1054_sim = model_LGBM_360.predict_proba(X_test_1054_sim)[:, 1]
test_preds_LGBM_1980_1054_sim = model_LGBM_1980.predict_proba(X_test_1054_sim)[:, 1]
test_preds_LGBM_3780_1054_sim = model_LGBM_3780.predict_proba(X_test_1054_sim)[:, 1]
test_preds_LGBM_5580_1054_sim = model_LGBM_5580.predict_proba(X_test_1054_sim)[:, 1]

In [ ]:
# For 360 DNNs, run forward pass and extract softmax scores
X_test_tensor_68 = torch.tensor(X_test_68, dtype=torch.float32).to(device)
X_test_tensor_374 = torch.tensor(X_test_374, dtype=torch.float32).to(device)
X_test_tensor_714 = torch.tensor(X_test_714, dtype=torch.float32).to(device)
X_test_tensor_1054 = torch.tensor(X_test_1054, dtype=torch.float32).to(device)
X_test_tensor_7382 = torch.tensor(X_test_7382, dtype=torch.float32).to(device)

X_test_tensor_68_sim = torch.tensor(X_test_68_sim, dtype=torch.float32).to(device)
X_test_tensor_374_sim = torch.tensor(X_test_374_sim, dtype=torch.float32).to(device)
X_test_tensor_714_sim = torch.tensor(X_test_714_sim, dtype=torch.float32).to(device)
X_test_tensor_1054_sim = torch.tensor(X_test_1054_sim, dtype=torch.float32).to(device)

In [ ]:
# For Cost-sensitive DNN setting
# X_test structure: 34 +34, 34 + 340, 34 + 680, 34 + 1020, 34 + 1364, 34 + 7348  
model_DNN_360.eval()
with torch.no_grad():
    test_preds_DNN_360_68 = torch.softmax(model_DNN_360(X_test_tensor_68), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_374 = torch.softmax(model_DNN_360(X_test_tensor_374), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_714 = torch.softmax(model_DNN_360(X_test_tensor_714), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_1054 = torch.softmax(model_DNN_360(X_test_tensor_1054), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_7382 = torch.softmax(model_DNN_360(X_test_tensor_7382), dim=1)[:, 1].cpu().numpy()
    
    test_preds_DNN_360_68_sim = torch.softmax(model_DNN_360(X_test_tensor_68_sim), dim=1)[:, 1].cpu().numpy() 
    test_preds_DNN_360_374_sim = torch.softmax(model_DNN_360(X_test_tensor_374_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_714_sim = torch.softmax(model_DNN_360(X_test_tensor_714_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_1054_sim = torch.softmax(model_DNN_360(X_test_tensor_1054_sim), dim=1)[:, 1].cpu().numpy()
    

model_DNN_1980.eval()
with torch.no_grad():
    test_preds_DNN_1980_68 = torch.softmax(model_DNN_1980(X_test_tensor_68), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_374 = torch.softmax(model_DNN_1980(X_test_tensor_374), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_714 = torch.softmax(model_DNN_1980(X_test_tensor_714), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_1054 = torch.softmax(model_DNN_1980(X_test_tensor_1054), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_7382 = torch.softmax(model_DNN_1980(X_test_tensor_7382), dim=1)[:, 1].cpu().numpy()
    
    test_preds_DNN_1980_68_sim = torch.softmax(model_DNN_1980(X_test_tensor_68_sim), dim=1)[:, 1].cpu().numpy() 
    test_preds_DNN_1980_374_sim = torch.softmax(model_DNN_1980(X_test_tensor_374_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_714_sim = torch.softmax(model_DNN_1980(X_test_tensor_714_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_1054_sim = torch.softmax(model_DNN_1980(X_test_tensor_1054_sim), dim=1)[:, 1].cpu().numpy()


model_DNN_3780.eval()
with torch.no_grad():
    test_preds_DNN_3780_68 = torch.softmax(model_DNN_3780(X_test_tensor_68), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_374 = torch.softmax(model_DNN_3780(X_test_tensor_374), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_714 = torch.softmax(model_DNN_3780(X_test_tensor_714), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_1054 = torch.softmax(model_DNN_3780(X_test_tensor_1054), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_7382 = torch.softmax(model_DNN_3780(X_test_tensor_7382), dim=1)[:, 1].cpu().numpy()
    
    test_preds_DNN_3780_68_sim = torch.softmax(model_DNN_3780(X_test_tensor_68_sim), dim=1)[:, 1].cpu().numpy() 
    test_preds_DNN_3780_374_sim = torch.softmax(model_DNN_3780(X_test_tensor_374_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_714_sim = torch.softmax(model_DNN_3780(X_test_tensor_714_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_1054_sim = torch.softmax(model_DNN_3780(X_test_tensor_1054_sim), dim=1)[:, 1].cpu().numpy()

model_DNN_5580.eval()
with torch.no_grad():
    test_preds_DNN_5580_68 = torch.softmax(model_DNN_5580(X_test_tensor_68), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_374 = torch.softmax(model_DNN_5580(X_test_tensor_374), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_714 = torch.softmax(model_DNN_5580(X_test_tensor_714), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_1054 = torch.softmax(model_DNN_5580(X_test_tensor_1054), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_7382 = torch.softmax(model_DNN_5580(X_test_tensor_7382), dim=1)[:, 1].cpu().numpy()
    
    test_preds_DNN_5580_68_sim = torch.softmax(model_DNN_5580(X_test_tensor_68_sim), dim=1)[:, 1].cpu().numpy() 
    test_preds_DNN_5580_374_sim = torch.softmax(model_DNN_5580(X_test_tensor_374_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_714_sim = torch.softmax(model_DNN_5580(X_test_tensor_714_sim), dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_1054_sim = torch.softmax(model_DNN_5580(X_test_tensor_1054_sim), dim=1)[:, 1].cpu().numpy()

In [ ]:
# Stack models predictions for testing set

# Regular
test_preds_stack_360_68 = np.column_stack([test_preds_RF_360_68, test_preds_LGBM_360_68, test_preds_DNN_360_68])
test_preds_stack_1980_68 = np.column_stack([test_preds_RF_1980_68, test_preds_LGBM_1980_68, test_preds_DNN_1980_68])
test_preds_stack_3780_68 = np.column_stack([test_preds_RF_3780_68, test_preds_LGBM_3780_68, test_preds_DNN_3780_68])
test_preds_stack_5580_68 = np.column_stack([test_preds_RF_5580_68, test_preds_LGBM_5580_68, test_preds_DNN_5580_68])

test_preds_stack_360_374 = np.column_stack([test_preds_RF_360_374, test_preds_LGBM_360_374, test_preds_DNN_360_374])
test_preds_stack_1980_374 = np.column_stack([test_preds_RF_1980_374, test_preds_LGBM_1980_374, test_preds_DNN_1980_374])
test_preds_stack_3780_374 = np.column_stack([test_preds_RF_3780_374, test_preds_LGBM_3780_374, test_preds_DNN_3780_374])
test_preds_stack_5580_374 = np.column_stack([test_preds_RF_5580_374, test_preds_LGBM_5580_374, test_preds_DNN_5580_374])

test_preds_stack_360_714 = np.column_stack([test_preds_RF_360_714, test_preds_LGBM_360_714, test_preds_DNN_360_714])
test_preds_stack_1980_714 = np.column_stack([test_preds_RF_1980_714, test_preds_LGBM_1980_714, test_preds_DNN_1980_714])
test_preds_stack_3780_714 = np.column_stack([test_preds_RF_3780_714, test_preds_LGBM_3780_714, test_preds_DNN_3780_714])
test_preds_stack_5580_714 = np.column_stack([test_preds_RF_5580_714, test_preds_LGBM_5580_714, test_preds_DNN_5580_714])

test_preds_stack_360_1054 = np.column_stack([test_preds_RF_360_1054, test_preds_LGBM_360_1054, test_preds_DNN_360_1054])
test_preds_stack_1980_1054 = np.column_stack([test_preds_RF_1980_1054, test_preds_LGBM_1980_1054, test_preds_DNN_1980_1054])
test_preds_stack_3780_1054 = np.column_stack([test_preds_RF_3780_1054, test_preds_LGBM_3780_1054, test_preds_DNN_3780_1054])
test_preds_stack_5580_1054 = np.column_stack([test_preds_RF_5580_1054, test_preds_LGBM_5580_1054, test_preds_DNN_5580_1054])

test_preds_stack_360_7382 = np.column_stack([test_preds_RF_360_7382, test_preds_LGBM_360_7382, test_preds_DNN_360_7382])
test_preds_stack_1980_7382 = np.column_stack([test_preds_RF_1980_7382, test_preds_LGBM_1980_7382, test_preds_DNN_1980_7382])
test_preds_stack_3780_7382 = np.column_stack([test_preds_RF_3780_7382, test_preds_LGBM_3780_7382, test_preds_DNN_3780_7382])
test_preds_stack_5580_7382 = np.column_stack([test_preds_RF_5580_7382, test_preds_LGBM_5580_7382, test_preds_DNN_5580_7382])

# Simulation
test_preds_stack_360_68_sim = np.column_stack([test_preds_RF_360_68_sim, test_preds_LGBM_360_68_sim, test_preds_DNN_360_68_sim])
test_preds_stack_1980_68_sim = np.column_stack([test_preds_RF_1980_68_sim, test_preds_LGBM_1980_68_sim, test_preds_DNN_1980_68_sim])
test_preds_stack_3780_68_sim = np.column_stack([test_preds_RF_3780_68_sim, test_preds_LGBM_3780_68_sim, test_preds_DNN_3780_68_sim])
test_preds_stack_5580_68_sim = np.column_stack([test_preds_RF_5580_68_sim, test_preds_LGBM_5580_68_sim, test_preds_DNN_5580_68_sim])

test_preds_stack_360_374_sim = np.column_stack([test_preds_RF_360_374_sim, test_preds_LGBM_360_374_sim, test_preds_DNN_360_374_sim])
test_preds_stack_1980_374_sim = np.column_stack([test_preds_RF_1980_374_sim, test_preds_LGBM_1980_374_sim, test_preds_DNN_1980_374_sim])
test_preds_stack_3780_374_sim = np.column_stack([test_preds_RF_3780_374_sim, test_preds_LGBM_3780_374_sim, test_preds_DNN_3780_374_sim])
test_preds_stack_5580_374_sim = np.column_stack([test_preds_RF_5580_374_sim, test_preds_LGBM_5580_374_sim, test_preds_DNN_5580_374_sim])

test_preds_stack_360_714_sim = np.column_stack([test_preds_RF_360_714_sim, test_preds_LGBM_360_714_sim, test_preds_DNN_360_714_sim])
test_preds_stack_1980_714_sim = np.column_stack([test_preds_RF_1980_714_sim, test_preds_LGBM_1980_714_sim, test_preds_DNN_1980_714_sim])
test_preds_stack_3780_714_sim = np.column_stack([test_preds_RF_3780_714_sim, test_preds_LGBM_3780_714_sim, test_preds_DNN_3780_714_sim])
test_preds_stack_5580_714_sim = np.column_stack([test_preds_RF_5580_714_sim, test_preds_LGBM_5580_714_sim, test_preds_DNN_5580_714_sim])

test_preds_stack_360_1054_sim = np.column_stack([test_preds_RF_360_1054_sim, test_preds_LGBM_360_1054_sim, test_preds_DNN_360_1054_sim])
test_preds_stack_1980_1054_sim = np.column_stack([test_preds_RF_1980_1054_sim, test_preds_LGBM_1980_1054_sim, test_preds_DNN_1980_1054_sim])
test_preds_stack_3780_1054_sim = np.column_stack([test_preds_RF_3780_1054_sim, test_preds_LGBM_3780_1054_sim, test_preds_DNN_3780_1054_sim])
test_preds_stack_5580_1054_sim = np.column_stack([test_preds_RF_5580_1054_sim, test_preds_LGBM_5580_1054_sim, test_preds_DNN_5580_1054_sim])

In [ ]:
# Predict final probabilities from LR meta-model

# Regular
final_probs_LR_360_68 = meta_model_LR_360.predict_proba(test_preds_stack_360_68)[:, 1]
final_probs_LR_1980_68 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_68)[:, 1]
final_probs_LR_3780_68 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_68)[:, 1]
final_probs_LR_5580_68 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_68)[:, 1]

final_probs_LR_360_374 = meta_model_LR_360.predict_proba(test_preds_stack_360_374)[:, 1]
final_probs_LR_1980_374 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_374)[:, 1]
final_probs_LR_3780_374 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_374)[:, 1]
final_probs_LR_5580_374 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_374)[:, 1]

final_probs_LR_360_714 = meta_model_LR_360.predict_proba(test_preds_stack_360_714)[:, 1]
final_probs_LR_1980_714 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_714)[:, 1]
final_probs_LR_3780_714 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_714)[:, 1]
final_probs_LR_5580_714 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_714)[:, 1]

final_probs_LR_360_1054 = meta_model_LR_360.predict_proba(test_preds_stack_360_1054)[:, 1]
final_probs_LR_1980_1054 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_1054)[:, 1]
final_probs_LR_3780_1054 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_1054)[:, 1]
final_probs_LR_5580_1054 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_1054)[:, 1]

final_probs_LR_360_7382 = meta_model_LR_360.predict_proba(test_preds_stack_360_7382)[:, 1]
final_probs_LR_1980_7382 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_7382)[:, 1]
final_probs_LR_3780_7382 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_7382)[:, 1]
final_probs_LR_5580_7382 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_7382)[:, 1]


# Simulation
final_probs_LR_360_68_sim = meta_model_LR_360.predict_proba(test_preds_stack_360_68_sim)[:, 1]
final_probs_LR_1980_68_sim = meta_model_LR_1980.predict_proba(test_preds_stack_1980_68_sim)[:, 1]
final_probs_LR_3780_68_sim = meta_model_LR_3780.predict_proba(test_preds_stack_3780_68_sim)[:, 1]
final_probs_LR_5580_68_sim = meta_model_LR_5580.predict_proba(test_preds_stack_5580_68_sim)[:, 1]

final_probs_LR_360_374_sim = meta_model_LR_360.predict_proba(test_preds_stack_360_374_sim)[:, 1]
final_probs_LR_1980_374_sim = meta_model_LR_1980.predict_proba(test_preds_stack_1980_374_sim)[:, 1]
final_probs_LR_3780_374_sim = meta_model_LR_3780.predict_proba(test_preds_stack_3780_374_sim)[:, 1]
final_probs_LR_5580_374_sim = meta_model_LR_5580.predict_proba(test_preds_stack_5580_374_sim)[:, 1]

final_probs_LR_360_714_sim = meta_model_LR_360.predict_proba(test_preds_stack_360_714_sim)[:, 1]
final_probs_LR_1980_714_sim = meta_model_LR_1980.predict_proba(test_preds_stack_1980_714_sim)[:, 1]
final_probs_LR_3780_714_sim = meta_model_LR_3780.predict_proba(test_preds_stack_3780_714_sim)[:, 1]
final_probs_LR_5580_714_sim = meta_model_LR_5580.predict_proba(test_preds_stack_5580_714_sim)[:, 1]

final_probs_LR_360_1054_sim = meta_model_LR_360.predict_proba(test_preds_stack_360_1054_sim)[:, 1]
final_probs_LR_1980_1054_sim = meta_model_LR_1980.predict_proba(test_preds_stack_1980_1054_sim)[:, 1]
final_probs_LR_3780_1054_sim = meta_model_LR_3780.predict_proba(test_preds_stack_3780_1054_sim)[:, 1]
final_probs_LR_5580_1054_sim = meta_model_LR_5580.predict_proba(test_preds_stack_5580_1054_sim)[:, 1]


In [ ]:
# Set thresholds
# Regular
final_preds_LR_360_68 = (final_probs_LR_360_68 >= 0.5).astype(int)
final_preds_LR_1980_68 = (final_probs_LR_1980_68 >= 0.5).astype(int)
final_preds_LR_3780_68 = (final_probs_LR_3780_68 >= 0.5).astype(int)
final_preds_LR_5580_68 = (final_probs_LR_5580_68 >= 0.5).astype(int)

final_preds_LR_360_374 = (final_probs_LR_360_374 >= 0.5).astype(int)
final_preds_LR_1980_374 = (final_probs_LR_1980_374 >= 0.5).astype(int)
final_preds_LR_3780_374 = (final_probs_LR_3780_374 >= 0.5).astype(int)
final_preds_LR_5580_374 = (final_probs_LR_5580_374 >= 0.5).astype(int)

final_preds_LR_360_714 = (final_probs_LR_360_714 >= 0.5).astype(int)
final_preds_LR_1980_714 = (final_probs_LR_1980_714 >= 0.5).astype(int)
final_preds_LR_3780_714 = (final_probs_LR_3780_714 >= 0.5).astype(int)
final_preds_LR_5580_714 = (final_probs_LR_5580_714 >= 0.5).astype(int)

final_preds_LR_360_1054 = (final_probs_LR_360_1054 >= 0.5).astype(int)
final_preds_LR_1980_1054 = (final_probs_LR_1980_1054 >= 0.5).astype(int)
final_preds_LR_3780_1054 = (final_probs_LR_3780_1054 >= 0.5).astype(int)
final_preds_LR_5580_1054 = (final_probs_LR_5580_1054 >= 0.5).astype(int)

final_preds_LR_360_7382 = (final_probs_LR_360_7382 >= 0.5).astype(int)
final_preds_LR_1980_7382 = (final_probs_LR_1980_7382 >= 0.5).astype(int)
final_preds_LR_3780_7382 = (final_probs_LR_3780_7382 >= 0.5).astype(int)
final_preds_LR_5580_7382 = (final_probs_LR_5580_7382 >= 0.5).astype(int)


# Simulation
final_preds_LR_360_68_sim = (final_probs_LR_360_68_sim >= 0.5).astype(int)
final_preds_LR_1980_68_sim = (final_probs_LR_1980_68_sim >= 0.5).astype(int)
final_preds_LR_3780_68_sim = (final_probs_LR_3780_68_sim >= 0.5).astype(int)
final_preds_LR_5580_68_sim = (final_probs_LR_5580_68_sim >= 0.5).astype(int)

final_preds_LR_360_374_sim = (final_probs_LR_360_374_sim >= 0.5).astype(int)
final_preds_LR_1980_374_sim = (final_probs_LR_1980_374_sim >= 0.5).astype(int)
final_preds_LR_3780_374_sim = (final_probs_LR_3780_374_sim >= 0.5).astype(int)
final_preds_LR_5580_374_sim = (final_probs_LR_5580_374_sim >= 0.5).astype(int)

final_preds_LR_360_714_sim = (final_probs_LR_360_714_sim >= 0.5).astype(int)
final_preds_LR_1980_714_sim = (final_probs_LR_1980_714_sim >= 0.5).astype(int)
final_preds_LR_3780_714_sim = (final_probs_LR_3780_714_sim >= 0.5).astype(int)
final_preds_LR_5580_714_sim = (final_probs_LR_5580_714_sim >= 0.5).astype(int)

final_preds_LR_360_1054_sim = (final_probs_LR_360_1054_sim >= 0.5).astype(int)
final_preds_LR_1980_1054_sim = (final_probs_LR_1980_1054_sim >= 0.5).astype(int)
final_preds_LR_3780_1054_sim = (final_probs_LR_3780_1054_sim >= 0.5).astype(int)
final_preds_LR_5580_1054_sim = (final_probs_LR_5580_1054_sim >= 0.5).astype(int)

In [ ]:
# Report tables
# Regular
print("Stacking Ensemble AUC of 360_68:", roc_auc_score(y_test_68, final_probs_LR_360_68))
print("Stacking Ensemble AUC of 1980_68:", roc_auc_score(y_test_68, final_probs_LR_1980_68))
print("Stacking Ensemble AUC of 3780_68", roc_auc_score(y_test_68, final_probs_LR_3780_68))
print("Stacking Ensemble AUC of 5580_68:\n", roc_auc_score(y_test_68, final_probs_LR_5580_68))

print("Classification Report of 360_374:\n", classification_report(y_test_374, final_preds_LR_360_374))
print("Classification Report of 1980_374:\n", classification_report(y_test_374, final_preds_LR_1980_374))
print("Classification Report of 3780_374:\n", classification_report(y_test_374, final_preds_LR_3780_374))
print("Classification Report of 5580_374:\n", classification_report(y_test_374, final_preds_LR_5580_374))

print("Stacking Ensemble AUC of 360_714:", roc_auc_score(y_test_714, final_probs_LR_360_714))
print("Stacking Ensemble AUC of 1980_714:", roc_auc_score(y_test_714, final_probs_LR_1980_714))
print("Stacking Ensemble AUC of 3780_714:", roc_auc_score(y_test_714, final_probs_LR_3780_714))
print("Stacking Ensemble AUC of 5580_714:\n", roc_auc_score(y_test_714, final_probs_LR_5580_714))

print("Classification Report of 360_714:\n", classification_report(y_test_714, final_preds_LR_360_714))
print("Classification Report of 1980_714:\n", classification_report(y_test_714, final_preds_LR_1980_714))
print("Classification Report of 3780_714:\n", classification_report(y_test_714, final_preds_LR_3780_714))
print("Classification Report of 5580_714:\n", classification_report(y_test_714, final_preds_LR_5580_714))

print("Stacking Ensemble AUC of 360_1054:", roc_auc_score(y_test_1054, final_probs_LR_360_1054))
print("Stacking Ensemble AUC of 1980_1054:", roc_auc_score(y_test_1054, final_probs_LR_1980_1054))
print("Stacking Ensemble AUC of 3780_1054:", roc_auc_score(y_test_1054, final_probs_LR_3780_1054))
print("Stacking Ensemble AUC of 5580_1054:\n", roc_auc_score(y_test_1054, final_probs_LR_5580_1054))

print("Classification Report of 360_1054:\n", classification_report(y_test_1054, final_preds_LR_360_1054))
print("Classification Report of 1980_1054:\n", classification_report(y_test_1054, final_preds_LR_1980_1054))
print("Classification Report of 3780_1054:\n", classification_report(y_test_1054, final_preds_LR_3780_1054))
print("Classification Report of 5580_1054:\n", classification_report(y_test_1054, final_preds_LR_5580_1054))

print("Stacking Ensemble AUC of 360_7382:", roc_auc_score(y_test_7382, final_probs_LR_360_7382))
print("Stacking Ensemble AUC of 1980_7382:", roc_auc_score(y_test_7382, final_probs_LR_1980_7382))
print("Stacking Ensemble AUC of 3780_7382:", roc_auc_score(y_test_7382, final_probs_LR_3780_7382))
print("Stacking Ensemble AUC of 5580_7382:\n", roc_auc_score(y_test_7382, final_probs_LR_5580_7382))

print("Classification Report of 360_7382:\n", classification_report(y_test_7382, final_preds_LR_360_7382))
print("Classification Report of 1980_7382:\n", classification_report(y_test_7382, final_preds_LR_1980_7382))
print("Classification Report of 3780_7382:\n", classification_report(y_test_7382, final_preds_LR_3780_7382))
print("Classification Report of 5580_7382:\n", classification_report(y_test_7382, final_preds_LR_5580_7382))

In [ ]:
# Simulation report
print("Stacking Ensemble AUC of 360_68_sim:", roc_auc_score(y_test_68, final_probs_LR_360_68_sim))
print("Stacking Ensemble AUC of 1980_68_sim:", roc_auc_score(y_test_68, final_probs_LR_1980_68_sim))
print("Stacking Ensemble AUC of 3780_68_sim:", roc_auc_score(y_test_68, final_probs_LR_3780_68_sim))
print("Stacking Ensemble AUC of 5580_68_sim:\n", roc_auc_score(y_test_68, final_probs_LR_5580_68_sim))

print("Classification Report of 360_374_sim:\n", classification_report(y_test_374, final_preds_LR_360_374_sim))
print("Classification Report of 1980_374_sim:\n", classification_report(y_test_374, final_preds_LR_1980_374_sim))
print("Classification Report of 3780_374_sim:\n", classification_report(y_test_374, final_preds_LR_3780_374_sim))
print("Classification Report of 5580_374_sim:\n", classification_report(y_test_374, final_preds_LR_5580_374_sim))

print("Stacking Ensemble AUC of 360_714_sim:", roc_auc_score(y_test_714, final_probs_LR_360_714_sim))
print("Stacking Ensemble AUC of 1980_714_sim:", roc_auc_score(y_test_714, final_probs_LR_1980_714_sim))
print("Stacking Ensemble AUC of 3780_714_sim:", roc_auc_score(y_test_714, final_probs_LR_3780_714_sim))
print("Stacking Ensemble AUC of 5580_714_sim:\n", roc_auc_score(y_test_714, final_probs_LR_5580_714_sim))

print("Classification Report of 360_714_sim:\n", classification_report(y_test_714, final_preds_LR_360_714_sim))
print("Classification Report of 1980_714_sim:\n", classification_report(y_test_714, final_preds_LR_1980_714_sim))
print("Classification Report of 3780_714_sim:\n", classification_report(y_test_714, final_preds_LR_3780_714_sim))
print("Classification Report of 5580_714_sim:\n", classification_report(y_test_714, final_preds_LR_5580_714_sim))

print("Stacking Ensemble AUC of 360_1054_sim:", roc_auc_score(y_test_1054, final_probs_LR_360_1054_sim))
print("Stacking Ensemble AUC of 1980_1054_sim:", roc_auc_score(y_test_1054, final_probs_LR_1980_1054_sim))
print("Stacking Ensemble AUC of 3780_1054_sim:", roc_auc_score(y_test_1054, final_probs_LR_3780_1054_sim))
print("Stacking Ensemble AUC of 5580_1054_sim:\n", roc_auc_score(y_test_1054, final_probs_LR_5580_1054_sim))

print("Classification Report of 360_1054_sim:\n", classification_report(y_test_1054, final_preds_LR_360_1054_sim))
print("Classification Report of 1980_1054_sim:\n", classification_report(y_test_1054, final_preds_LR_1980_1054_sim))
print("Classification Report of 3780_1054_sim:\n", classification_report(y_test_1054, final_preds_LR_3780_1054_sim))
print("Classification Report of 5580_1054_sim:\n", classification_report(y_test_1054, final_preds_LR_5580_1054_sim))


In [ ]:
# Re-defining the function since kernel reset might have cleared previous definitions

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_auc_score, confusion_matrix, f1_score, roc_curve
)

# Function to evaluate meta-learner with 95% CI and ROC plot
def evaluate_meta_model_with_ci(y_true, y_prob, threshold=0.1, n_bootstrap=1000, seed=42):
    np.random.seed(seed)
    y_pred = (y_prob >= threshold).astype(int)

    auc = roc_auc_score(y_true, y_prob)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if tp + fp > 0 else 0
    npv = tn / (tn + fn) if tn + fn > 0 else 0
    f1 = f1_score(y_true, y_pred)

    aucs, sens, specs, ppvs, npvs, f1s = [], [], [], [], [], []

    n = len(y_true)
    for _ in range(n_bootstrap):
        indices = np.random.choice(np.arange(n), size=n, replace=True)
        y_true_bs = y_true[indices]
        y_prob_bs = y_prob[indices]
        y_pred_bs = (y_prob_bs >= threshold).astype(int)

        try:
            aucs.append(roc_auc_score(y_true_bs, y_prob_bs))
            tn_b, fp_b, fn_b, tp_b = confusion_matrix(y_true_bs, y_pred_bs).ravel()
            sens.append(tp_b / (tp_b + fn_b))
            specs.append(tn_b / (tn_b + fp_b))
            ppvs.append(tp_b / (tp_b + fp_b) if tp_b + fp_b > 0 else 0)
            npvs.append(tn_b / (tn_b + fn_b) if tn_b + fn_b > 0 else 0)
            f1s.append(f1_score(y_true_bs, y_pred_bs))
        except:
            continue

    def ci(data):
        return np.mean(data), np.percentile(data, 2.5), np.percentile(data, 97.5)

    metrics = {
        'AUC': ci(aucs),
        'Sensitivity': ci(sens),
        'Specificity': ci(specs),
        'PPV': ci(ppvs),
        'NPV': ci(npvs),
        'F1-score': ci(f1s)
    }

    for metric, (mean, lower, upper) in metrics.items():
        print(f"{metric}: {mean:.4f} (95% CI: {lower:.4f} - {upper:.4f})")

    fpr, tpr, _ = roc_curve(y_true, y_prob)
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, label=f"AUC = {auc:.4f}")
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve of Stacking Meta-Learner")
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()
    
    return metrics


In [ ]:
metrics_360_68 = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_360_68, threshold=0.5)
metrics_1980_68 = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_1980_68, threshold=0.5)
metrics_3780_68 = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_3780_68, threshold=0.5)
metrics_5580_68 = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_5580_68, threshold=0.5)

metrics_360_374 = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_360_374, threshold=0.5)
metrics_1980_374 = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_1980_374, threshold=0.5)
metrics_3780_374 = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_3780_374, threshold=0.5)
metrics_5580_374 = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_5580_374, threshold=0.5)

metrics_360_714 = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_360_714, threshold=0.5)
metrics_1980_714 = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_1980_714, threshold=0.5)
metrics_3780_714 = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_3780_714, threshold=0.5)
metrics_5580_714 = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_5580_714, threshold=0.5)

metrics_360_1054 = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_360_1054, threshold=0.5)
metrics_1980_1054 = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_1980_1054, threshold=0.5)
metrics_3780_1054 = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_3780_1054, threshold=0.5)
metrics_5580_1054 = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_5580_1054, threshold=0.5)

metrics_360_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_360_7382, threshold=0.5)
metrics_1980_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_1980_7382, threshold=0.5)
metrics_3780_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_3780_7382, threshold=0.5)
metrics_5580_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_5580_7382, threshold=0.5)

metrics_360_68_sim = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_360_68_sim, threshold=0.5)
metrics_1980_68_sim = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_1980_68_sim, threshold=0.5)
metrics_3780_68_sim = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_3780_68_sim, threshold=0.5)
metrics_5580_68_sim = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_5580_68_sim, threshold=0.5)

metrics_360_374_sim = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_360_374_sim, threshold=0.5)
metrics_1980_374_sim = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_1980_374_sim, threshold=0.5)
metrics_3780_374_sim = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_3780_374_sim, threshold=0.5)
metrics_5580_374_sim = evaluate_meta_model_with_ci(y_test_374, final_probs_LR_5580_374_sim, threshold=0.5)

metrics_360_714_sim = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_360_714_sim, threshold=0.5)
metrics_1980_714_sim = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_1980_714_sim, threshold=0.5)
metrics_3780_714_sim = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_3780_714_sim, threshold=0.5)
metrics_5580_714_sim = evaluate_meta_model_with_ci(y_test_714, final_probs_LR_5580_714_sim, threshold=0.5)

metrics_360_1054_sim = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_360_1054_sim, threshold=0.5)
metrics_1980_1054_sim = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_1980_1054_sim, threshold=0.5)
metrics_3780_1054_sim = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_3780_1054_sim, threshold=0.5)
metrics_5580_1054_sim = evaluate_meta_model_with_ci(y_test_1054, final_probs_LR_5580_1054_sim, threshold=0.5)

In [ ]:
metrics_360_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_360_7382, threshold=0.5)

In [ ]:
metrics_1980_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_1980_7382, threshold=0.5)

In [ ]:
metrics_3780_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_3780_7382, threshold=0.5)

In [ ]:
metrics_5580_7382 = evaluate_meta_model_with_ci(y_test_7382, final_probs_LR_5580_7382, threshold=0.5)

In [ ]:
metrics_360_68 = evaluate_meta_model_with_ci(y_test_68, final_probs_LR_360_68, threshold=0.5)

In [ ]:
metrics_360_68_sim = evaluate_meta_model_with_ci(y_test_68_sim, final_probs_LR_360_68_sim, threshold=0.5)

<h2 style="color: yellow;"> External Validation Section</h2>
<p style="color: yellow;">1. Check if there is data leaking;<br> 2. Eliminate features that leak information. <br> 3. Run validation with models</p>

#################################################################################
# 1. Check if there is data leaking

In [ ]:
# Read eICU raw dataset
meningitis_val_raw = pd.read_csv('Meningitis_val_raw.csv')
nonmeningitis_val_raw = pd.read_csv('nonmeningitis_val_raw.csv')

In [ ]:
# Check whether there are meningitis related columns (starting with '32') in Meningitis external validation dataset
print("Checking for columns starting with '32' in meningitis_val_raw...")
print("="*60)

# Get all column names
all_columns_men_val = meningitis_val_raw.columns.tolist()

# Find columns that start with "32"
columns_starting_32_men_val = [col for col in all_columns_men_val if str(col).startswith(('320','321','322'))]

print(f"Total columns in dataset: {len(all_columns_men_val)}")
print(f"Columns starting with '32': {len(columns_starting_32_men_val)}")
print()

if columns_starting_32_men_val:
    print("Found columns starting with '32':")
    print("-" * 40)
    
    # Sort the columns for better readability
    columns_starting_32_sorted_men_val = sorted(columns_starting_32_men_val)
    
    for i, col in enumerate(columns_starting_32_sorted_men_val, 1):
        # Check how many patients have this code
        count = meningitis_val_raw[col].sum() if col in meningitis_val_raw.columns else 0
        percentage = (count / len(meningitis_val_raw)) * 100 if len(meningitis_val_raw) > 0 else 0
        
        # Check positive rate when this code is present
        if count > 0:
            pos_rate = meningitis_val_raw[meningitis_val_raw[col] == 1]['CLASSIFIER'].mean()
        else:
            pos_rate = 0
            
        print(f"{i:2d}. {str(col):<8} - Present in {int(count):4d} patients ({percentage:5.2f}%) - Pos rate: {pos_rate:.3f}")
    
    print()
    print("Detailed analysis of '32' codes:")
    print("-" * 50)
    
    # Group by length for better understanding
    length_groups = {}
    for col in columns_starting_32_sorted_men_val:
        length = len(str(col))
        if length not in length_groups:
            length_groups[length] = []
        length_groups[length].append(col)
    
    for length in sorted(length_groups.keys()):
        print(f"\nLength {length} codes ({len(length_groups[length])} codes):")
        for col in length_groups[length]:
            count = int(meningitis_val_raw[col].sum())
            percentage = (count / len(meningitis_val_raw)) * 100
            if count > 0:
                pos_rate = meningitis_val_raw[meningitis_val_raw[col] == 1]['CLASSIFIER'].mean()
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - Meningitis rate: {pos_rate:.3f}")
            else:
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - No patients with this code")

In [ ]:
# Check whether there are meningitis related columns (starting with '32') in non-meningitis external validation dataset
print("Checking for columns starting with '32' in nonmeningitis_val_raw...")
print("="*60)

# Get all column names
all_columns_nonmen_val = nonmeningitis_val_raw.columns.tolist()

# Find columns that start with "32"
columns_starting_32_nonmen_val = [col for col in all_columns_nonmen_val if str(col).startswith(('320','321','322'))]

print(f"Total columns in dataset: {len(all_columns_nonmen_val)}")
print(f"Columns starting with '32': {len(columns_starting_32_nonmen_val)}")
print()

if columns_starting_32_nonmen_val:
    print("Found columns starting with '32':")
    print("-" * 40)
    
    # Sort the columns for better readability
    columns_starting_32_sorted_nonmen_val = sorted(columns_starting_32_nonmen_val)
    
    for i, col in enumerate(columns_starting_32_sorted_nonmen_val, 1):
        # Check how many patients have this code
        count = nonmeningitis_val_raw[col].sum() if col in nonmeningitis_val_raw.columns else 0
        percentage = (count / len(nonmeningitis_val_raw)) * 100 if len(nonmeningitis_val_raw) > 0 else 0
        
        # Check positive rate when this code is present
        if count > 0:
            pos_rate = nonmeningitis_val_raw[nonmeningitis_val_raw[col] == 1]['CLASSIFIER'].mean()
        else:
            pos_rate = 0
            
        print(f"{i:2d}. {str(col):<8} - Present in {int(count):4d} patients ({percentage:5.2f}%) - Pos rate: {pos_rate:.3f}")
    
    print()
    print("Detailed analysis of '32' codes:")
    print("-" * 50)
    
    # Group by length for better understanding
    length_groups = {}
    for col in columns_starting_32_sorted_nonmen_val:
        length = len(str(col))
        if length not in length_groups:
            length_groups[length] = []
        length_groups[length].append(col)
    
    for length in sorted(length_groups.keys()):
        print(f"\nLength {length} codes ({len(length_groups[length])} codes):")
        for col in length_groups[length]:
            count = int(nonmeningitis_val_raw[col].sum())
            percentage = (count / len(nonmeningitis_val_raw)) * 100
            if count > 0:
                pos_rate = nonmeningitis_val_raw[nonmeningitis_val_raw[col] == 1]['CLASSIFIER'].mean()
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - Meningitis rate: {pos_rate:.3f}")
            else:
                print(f"  {str(col)}: {count} patients ({percentage:.2f}%) - No patients with this code")

In [ ]:
# # Create new dataset without meningitis code columns
meningitis_clean_val = meningitis_val_raw.drop(columns=columns_starting_32_men_val)

In [ ]:
# Check columns that start with "('320','321','322')"
all_columns_clean_men_val = meningitis_clean_val.columns.tolist()

columns_starting_32_clean_val = [col for col in all_columns_clean_men_val if str(col).startswith(('320','321','322'))]

print(f"Total columns in dataset: {len(all_columns_clean_men_val)}")
print(f"Columns starting with '('320','321','322')': {len(columns_starting_32_clean_val)}")
print()

In [ ]:
# Check if there is meningitis sample in nonmeningitis_val_raw
(nonmeningitis_val_raw["CLASSIFIER"] == 1).any()

In [ ]:
# Check shapes
print(meningitis_clean_val.shape)
print(nonmeningitis_val_raw.shape)
print(meningitis_clean_val.head)
print(nonmeningitis_val_raw.head)

In [ ]:
# Ramdonly extract 124*10 = 1240, 124*20 = 2480, 124*30 = 3720, 124*218 = 27032 from nonmeningitis_val_raw for multiple external validation
non_men_1240_val = nonmeningitis_val_raw.sample(n=1240, random_state=42)
non_men_2480_val = nonmeningitis_val_raw.sample(n=2480, random_state=42)
non_men_3720_val = nonmeningitis_val_raw.sample(n=3720, random_state=42)
non_men_27032_val = nonmeningitis_val_raw.sample(n=27032, random_state=42)

In [ ]:
print(non_men_1240_val)
print(non_men_2480_val)
print(non_men_3720_val)
print(non_men_27032_val)

In [ ]:
# Concatinate meningitis and nonmeningitis validation datasets
val_concat_1364 = pd.concat([meningitis_clean_val, non_men_1240_val], ignore_index = True)
val_concat_2604 = pd.concat([meningitis_clean_val, non_men_2480_val], ignore_index = True)
val_concat_3844 = pd.concat([meningitis_clean_val, non_men_3720_val], ignore_index = True)
val_concat_27156 = pd.concat([meningitis_clean_val, non_men_27032_val], ignore_index = True)

In [ ]:
# Drop SUBJECT_ID of validation sets
val_IDdrop_1364 = val_concat_1364.drop('SUBJECT_ID', axis = 1)
val_IDdrop_2604 = val_concat_2604.drop('SUBJECT_ID', axis = 1)
val_IDdrop_3844 = val_concat_3844.drop('SUBJECT_ID', axis = 1)
val_IDdrop_27156 = val_concat_27156.drop('SUBJECT_ID', axis = 1)

In [ ]:
# Shuffle the validation set to ensure random distribution
val_shuffled_1364 = val_IDdrop_1364.sample(frac=1, random_state=42).reset_index(drop=True)
val_shuffled_2604 = val_IDdrop_2604.sample(frac=1, random_state=42).reset_index(drop=True)
val_shuffled_3844 = val_IDdrop_3844.sample(frac=1, random_state=42).reset_index(drop=True)
val_shuffled_27156 = val_IDdrop_27156.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print(val_shuffled_1364.shape)
print(val_shuffled_2604.shape)
print(val_shuffled_3844.shape)
print(val_shuffled_27156.shape)

In [ ]:
print("\n" + "NaN values in validation data (1364 samples):\n", val_shuffled_1364.isnull().sum())
print("\n" + "NaN values in validation data (2604 samples):\n", val_shuffled_2604.isnull().sum())
print("\n" + "NaN values in validation data (3844 samples):\n", val_shuffled_3844.isnull().sum())
print("\n" + "NaN values in validation data (27156 samples):\n", val_shuffled_27156.isnull().sum())

In [ ]:
# Replace NaN values with 0 in each dataset
df_val_1364 = val_shuffled_1364.fillna(0)
df_val_2064 = val_shuffled_2604.fillna(0)
df_val_3844 = val_shuffled_3844.fillna(0)
df_val_27156 = val_shuffled_27156.fillna(0)

# Verify if NaN are replaced
print("\n" + "NaN values in validation data (1364 samples):\n", df_val_1364.isnull().sum())
print("\n" + "NaN values in validation data (2604 samples):\n", df_val_2064.isnull().sum())
print("\n" + "NaN values in validation data (3844 samples):\n", df_val_3844.isnull().sum())
print("\n" + "NaN values in validation data (27156 samples):\n", df_val_27156.isnull().sum())

In [ ]:
# Check duplicate column names in validation datasets
print(df_val_1364.columns[df_val_1364.columns.duplicated()].unique()) 
print(df_val_2064.columns[df_val_2064.columns.duplicated()].unique()) 
print(df_val_3844.columns[df_val_3844.columns.duplicated()].unique()) 
print(df_val_27156.columns[df_val_27156.columns.duplicated()].unique()) 

In [ ]:
# Use df_training
# Extract all column names from the training data as reference
train_columns = df_training.columns.tolist()
print(f"Training data has {len(train_columns)} columns")

In [ ]:
# Separate the target column (CLASSIFIER) from feature columns
# We align features only — the target column is handled separately
target_col = 'CLASSIFIER'
train_feature_cols = [col for col in train_columns if col != target_col]
print(f"Training has {len(train_feature_cols)} feature columns + '{target_col}'")

In [ ]:
def align_to_train(df_val, train_feature_cols, target_col='CLASSIFIER'):
    """
    Align a validation DataFrame so its features match training exactly.

    Steps:
      A) Add missing columns (in training but not in validation) filled with 0
      B) Drop extra columns (in validation but not in training)
      C) Reorder to match training column order
      D) Re-attach the target column at the end
    """
    # --- Step A: Find feature columns missing in validation ---
    val_feature_cols = [c for c in df_val.columns if c != target_col]
    missing_cols = set(train_feature_cols) - set(val_feature_cols)
    print(f"  Missing columns to add (filled with 0): {len(missing_cols)}")

    # --- Step B: Find extra columns in validation that training doesn't have ---
    extra_cols = set(val_feature_cols) - set(train_feature_cols)
    print(f"  Extra columns to drop: {len(extra_cols)}")

    # --- Step C: Add missing columns with 0 ---
    for col in missing_cols:
        df_val[col] = 0

    # --- Step D: Select only training features (in order) + target column ---
    df_val = df_val[train_feature_cols + [target_col]]

    return df_val

In [ ]:
# Align df_val_1364
print("Aligning df_val_1364:")
df_val_1364 = align_to_train(df_val_1364, train_feature_cols)
print(f"  Result shape: {df_val_1364.shape}\n")

In [ ]:
# Align df_val_2064
print("Aligning df_val_2064:")
df_val_2064 = align_to_train(df_val_2064, train_feature_cols)
print(f"  Result shape: {df_val_2064.shape}\n")

In [ ]:
# Align df_val_3844
print("Aligning df_val_3844:")
df_val_3844 = align_to_train(df_val_3844, train_feature_cols)
print(f"  Result shape: {df_val_3844.shape}\n")

In [ ]:
# Align df_val_27156
print("Aligning df_val_27156:")
df_val_27156 = align_to_train(df_val_27156, train_feature_cols)
print(f"  Result shape: {df_val_27156.shape}\n")

In [ ]:
# Align df_val_2064
print("Aligning df_val_2064:")
df_val_2064 = align_to_train(df_val_2064, train_feature_cols)
print(f"  Result shape: {df_val_2064.shape}\n")

In [ ]:
# Align df_val_3844
print("Aligning df_val_3844:")
df_val_3844 = align_to_train(df_val_3844, train_feature_cols)
print(f"  Result shape: {df_val_3844.shape}\n")

In [ ]:
# Align df_val_27156
print("Aligning df_val_27156:")
df_val_27156 = align_to_train(df_val_27156, train_feature_cols)
print(f"  Result shape: {df_val_27156.shape}\n")

In [ ]:
# Reorder validation columns to exactly match df_training column order
df_val_1364 = df_val_1364[df_training.columns]
df_val_2064 = df_val_2064[df_training.columns]
df_val_3844 = df_val_3844[df_training.columns]
df_val_27156 = df_val_27156[df_training.columns]

print("✓ All validation sets reordered to match training column order")

In [ ]:
# Verify all aligned shapes and column order match training
for name, df_val in [("df_val_1364", df_val_1364),
                      ("df_val_2064", df_val_2064),
                      ("df_val_3844", df_val_3844),
                      ("df_val_27156", df_val_27156)]:

    assert df_val.shape[1] == df_training.shape[1], \
        f"{name} has {df_val.shape[1]} cols, expected {df_training.shape[1]}"

    assert list(df_val.columns) == list(df_training.columns), \
        f"{name} columns do not match training columns"

    print(f"✓ {name} aligned — shape: {df_val.shape}")

In [ ]:
############## Extract validation features and labels #################

In [ ]:
# ──────────────────────────────────────────────────────────────────
# Step 1: Separate features (X) and labels (y) for each validation set
# Same pattern as: X_test_68 = df_testing_68.drop('CLASSIFIER', axis=1).values
# ──────────────────────────────────────────────────────────────────

X_val_1364  = df_val_1364.drop('CLASSIFIER', axis=1).values
X_val_2064  = df_val_2064.drop('CLASSIFIER', axis=1).values
X_val_3844  = df_val_3844.drop('CLASSIFIER', axis=1).values
X_val_27156 = df_val_27156.drop('CLASSIFIER', axis=1).values

y_val_1364  = df_val_1364['CLASSIFIER'].values
y_val_2064  = df_val_2064['CLASSIFIER'].values
y_val_3844  = df_val_3844['CLASSIFIER'].values
y_val_27156 = df_val_27156['CLASSIFIER'].values

print(f"X_val_1364:  {X_val_1364.shape},  y_val_1364:  {y_val_1364.shape}")
print(f"X_val_2064:  {X_val_2064.shape},  y_val_2064:  {y_val_2064.shape}")
print(f"X_val_3844:  {X_val_3844.shape},  y_val_3844:  {y_val_3844.shape}")
print(f"X_val_27156: {X_val_27156.shape}, y_val_27156: {y_val_27156.shape}")

In [ ]:
############## Random Forest predictions on all validation sets #################

In [ ]:
# ──────────────────────────────────────────────────────────────────
# Step 2: Get RF base model predictions (probability of class 1)
# Each trained model variant (360, 1980, 3780, 5580) predicts on
# each validation set → 4 models × 4 val sets = 16 prediction arrays
# ──────────────────────────────────────────────────────────────────

# RF predictions — model trained on 360 samples
test_preds_RF_360_1364  = model_RF_360.predict_proba(X_val_1364)[:, 1]
test_preds_RF_360_2064  = model_RF_360.predict_proba(X_val_2064)[:, 1]
test_preds_RF_360_3844  = model_RF_360.predict_proba(X_val_3844)[:, 1]
test_preds_RF_360_27156 = model_RF_360.predict_proba(X_val_27156)[:, 1]

# RF predictions — model trained on 1980 samples
test_preds_RF_1980_1364  = model_RF_1980.predict_proba(X_val_1364)[:, 1]
test_preds_RF_1980_2064  = model_RF_1980.predict_proba(X_val_2064)[:, 1]
test_preds_RF_1980_3844  = model_RF_1980.predict_proba(X_val_3844)[:, 1]
test_preds_RF_1980_27156 = model_RF_1980.predict_proba(X_val_27156)[:, 1]

# RF predictions — model trained on 3780 samples
test_preds_RF_3780_1364  = model_RF_3780.predict_proba(X_val_1364)[:, 1]
test_preds_RF_3780_2064  = model_RF_3780.predict_proba(X_val_2064)[:, 1]
test_preds_RF_3780_3844  = model_RF_3780.predict_proba(X_val_3844)[:, 1]
test_preds_RF_3780_27156 = model_RF_3780.predict_proba(X_val_27156)[:, 1]

# RF predictions — model trained on 5580 samples
test_preds_RF_5580_1364  = model_RF_5580.predict_proba(X_val_1364)[:, 1]
test_preds_RF_5580_2064  = model_RF_5580.predict_proba(X_val_2064)[:, 1]
test_preds_RF_5580_3844  = model_RF_5580.predict_proba(X_val_3844)[:, 1]
test_preds_RF_5580_27156 = model_RF_5580.predict_proba(X_val_27156)[:, 1]

print("✓ RF predictions complete for all validation sets")

In [ ]:
############## LightGBM predictions on all validation sets #################

In [ ]:
# ──────────────────────────────────────────────────────────────────
# Step 3: Get LightGBM base model predictions (probability of class 1)
# Same approach: 4 models × 4 val sets = 16 prediction arrays
# ──────────────────────────────────────────────────────────────────

# LGBM predictions — model trained on 360 samples
test_preds_LGBM_360_1364  = model_LGBM_360.predict_proba(X_val_1364)[:, 1]
test_preds_LGBM_360_2064  = model_LGBM_360.predict_proba(X_val_2064)[:, 1]
test_preds_LGBM_360_3844  = model_LGBM_360.predict_proba(X_val_3844)[:, 1]
test_preds_LGBM_360_27156 = model_LGBM_360.predict_proba(X_val_27156)[:, 1]

# LGBM predictions — model trained on 1980 samples
test_preds_LGBM_1980_1364  = model_LGBM_1980.predict_proba(X_val_1364)[:, 1]
test_preds_LGBM_1980_2064  = model_LGBM_1980.predict_proba(X_val_2064)[:, 1]
test_preds_LGBM_1980_3844  = model_LGBM_1980.predict_proba(X_val_3844)[:, 1]
test_preds_LGBM_1980_27156 = model_LGBM_1980.predict_proba(X_val_27156)[:, 1]

# LGBM predictions — model trained on 3780 samples
test_preds_LGBM_3780_1364  = model_LGBM_3780.predict_proba(X_val_1364)[:, 1]
test_preds_LGBM_3780_2064  = model_LGBM_3780.predict_proba(X_val_2064)[:, 1]
test_preds_LGBM_3780_3844  = model_LGBM_3780.predict_proba(X_val_3844)[:, 1]
test_preds_LGBM_3780_27156 = model_LGBM_3780.predict_proba(X_val_27156)[:, 1]

# LGBM predictions — model trained on 5580 samples
test_preds_LGBM_5580_1364  = model_LGBM_5580.predict_proba(X_val_1364)[:, 1]
test_preds_LGBM_5580_2064  = model_LGBM_5580.predict_proba(X_val_2064)[:, 1]
test_preds_LGBM_5580_3844  = model_LGBM_5580.predict_proba(X_val_3844)[:, 1]
test_preds_LGBM_5580_27156 = model_LGBM_5580.predict_proba(X_val_27156)[:, 1]

print("✓ LightGBM predictions complete for all validation sets")

In [ ]:
############## DNN predictions on all validation sets #################

In [ ]:
# ──────────────────────────────────────────────────────────────────
# Step 4: Get DNN base model predictions
# DNN requires: (a) convert to PyTorch tensor, (b) forward pass,
# (c) apply softmax, (d) extract class-1 probability
# ──────────────────────────────────────────────────────────────────

# Convert validation features to PyTorch tensors on the correct device
X_val_tensor_1364  = torch.tensor(X_val_1364,  dtype=torch.float32).to(device)
X_val_tensor_2064  = torch.tensor(X_val_2064,  dtype=torch.float32).to(device)
X_val_tensor_3844  = torch.tensor(X_val_3844,  dtype=torch.float32).to(device)
X_val_tensor_27156 = torch.tensor(X_val_27156, dtype=torch.float32).to(device)

# DNN predictions — using all 4 model variants (360, 1980, 3780, 5580)
# Put each model in eval mode and disable gradient computation
for model_name, model_obj in [("DNN_360", model_DNN_360),
                               ("DNN_1980", model_DNN_1980),
                               ("DNN_3780", model_DNN_3780),
                               ("DNN_5580", model_DNN_5580)]:
    model_obj.eval()

with torch.no_grad():
    # model_DNN_360 predictions
    test_preds_DNN_360_1364  = torch.softmax(model_DNN_360(X_val_tensor_1364),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_2064  = torch.softmax(model_DNN_360(X_val_tensor_2064),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_3844  = torch.softmax(model_DNN_360(X_val_tensor_3844),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_360_27156 = torch.softmax(model_DNN_360(X_val_tensor_27156), dim=1)[:, 1].cpu().numpy()

    # model_DNN_1980 predictions
    test_preds_DNN_1980_1364  = torch.softmax(model_DNN_1980(X_val_tensor_1364),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_2064  = torch.softmax(model_DNN_1980(X_val_tensor_2064),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_3844  = torch.softmax(model_DNN_1980(X_val_tensor_3844),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_1980_27156 = torch.softmax(model_DNN_1980(X_val_tensor_27156), dim=1)[:, 1].cpu().numpy()

    # model_DNN_3780 predictions
    test_preds_DNN_3780_1364  = torch.softmax(model_DNN_3780(X_val_tensor_1364),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_2064  = torch.softmax(model_DNN_3780(X_val_tensor_2064),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_3844  = torch.softmax(model_DNN_3780(X_val_tensor_3844),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_3780_27156 = torch.softmax(model_DNN_3780(X_val_tensor_27156), dim=1)[:, 1].cpu().numpy()

    # model_DNN_5580 predictions
    test_preds_DNN_5580_1364  = torch.softmax(model_DNN_5580(X_val_tensor_1364),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_2064  = torch.softmax(model_DNN_5580(X_val_tensor_2064),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_3844  = torch.softmax(model_DNN_5580(X_val_tensor_3844),  dim=1)[:, 1].cpu().numpy()
    test_preds_DNN_5580_27156 = torch.softmax(model_DNN_5580(X_val_tensor_27156), dim=1)[:, 1].cpu().numpy()

print("✓ DNN predictions complete for all validation sets")

In [ ]:
############## Stack base model predictions into meta-features #################

In [ ]:
# ──────────────────────────────────────────────────────────────────
# Step 5: Stack RF + LGBM + DNN predictions into 3-column matrices
# Each row = one sample; each column = one base model's P(class=1)
# These become the input features for the LR meta-learner
# ──────────────────────────────────────────────────────────────────

# Stacked predictions for val_1364
test_preds_stack_360_1364  = np.column_stack([test_preds_RF_360_1364,  test_preds_LGBM_360_1364,  test_preds_DNN_360_1364])
test_preds_stack_1980_1364 = np.column_stack([test_preds_RF_1980_1364, test_preds_LGBM_1980_1364, test_preds_DNN_1980_1364])
test_preds_stack_3780_1364 = np.column_stack([test_preds_RF_3780_1364, test_preds_LGBM_3780_1364, test_preds_DNN_3780_1364])
test_preds_stack_5580_1364 = np.column_stack([test_preds_RF_5580_1364, test_preds_LGBM_5580_1364, test_preds_DNN_5580_1364])

# Stacked predictions for val_2064
test_preds_stack_360_2064  = np.column_stack([test_preds_RF_360_2064,  test_preds_LGBM_360_2064,  test_preds_DNN_360_2064])
test_preds_stack_1980_2064 = np.column_stack([test_preds_RF_1980_2064, test_preds_LGBM_1980_2064, test_preds_DNN_1980_2064])
test_preds_stack_3780_2064 = np.column_stack([test_preds_RF_3780_2064, test_preds_LGBM_3780_2064, test_preds_DNN_3780_2064])
test_preds_stack_5580_2064 = np.column_stack([test_preds_RF_5580_2064, test_preds_LGBM_5580_2064, test_preds_DNN_5580_2064])

# Stacked predictions for val_3844
test_preds_stack_360_3844  = np.column_stack([test_preds_RF_360_3844,  test_preds_LGBM_360_3844,  test_preds_DNN_360_3844])
test_preds_stack_1980_3844 = np.column_stack([test_preds_RF_1980_3844, test_preds_LGBM_1980_3844, test_preds_DNN_1980_3844])
test_preds_stack_3780_3844 = np.column_stack([test_preds_RF_3780_3844, test_preds_LGBM_3780_3844, test_preds_DNN_3780_3844])
test_preds_stack_5580_3844 = np.column_stack([test_preds_RF_5580_3844, test_preds_LGBM_5580_3844, test_preds_DNN_5580_3844])

# Stacked predictions for val_27156
test_preds_stack_360_27156  = np.column_stack([test_preds_RF_360_27156,  test_preds_LGBM_360_27156,  test_preds_DNN_360_27156])
test_preds_stack_1980_27156 = np.column_stack([test_preds_RF_1980_27156, test_preds_LGBM_1980_27156, test_preds_DNN_1980_27156])
test_preds_stack_3780_27156 = np.column_stack([test_preds_RF_3780_27156, test_preds_LGBM_3780_27156, test_preds_DNN_3780_27156])
test_preds_stack_5580_27156 = np.column_stack([test_preds_RF_5580_27156, test_preds_LGBM_5580_27156, test_preds_DNN_5580_27156])

print("✓ All base model predictions stacked into meta-features")
print(f"   Example shape: {test_preds_stack_360_1364.shape}")  # Should be (n_samples, 3)

In [ ]:
############## Final ensemble predictions from meta-learner #################

In [ ]:
# ──────────────────────────────────────────────────────────────────
# Step 6: Feed stacked predictions into the LR meta-learner
# Each meta-model variant (360, 1980, 3780, 5580) predicts on each
# validation set. Output: probability of class 1 → binary prediction
# ──────────────────────────────────────────────────────────────────

# --- val_1364 ---
final_probs_LR_360_1364  = meta_model_LR_360.predict_proba(test_preds_stack_360_1364)[:, 1]
final_probs_LR_1980_1364 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_1364)[:, 1]
final_probs_LR_3780_1364 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_1364)[:, 1]
final_probs_LR_5580_1364 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_1364)[:, 1]

final_preds_LR_360_1364  = (final_probs_LR_360_1364  >= 0.5).astype(int)
final_preds_LR_1980_1364 = (final_probs_LR_1980_1364 >= 0.5).astype(int)
final_preds_LR_3780_1364 = (final_probs_LR_3780_1364 >= 0.5).astype(int)
final_preds_LR_5580_1364 = (final_probs_LR_5580_1364 >= 0.5).astype(int)

# --- val_2064 ---
final_probs_LR_360_2064  = meta_model_LR_360.predict_proba(test_preds_stack_360_2064)[:, 1]
final_probs_LR_1980_2064 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_2064)[:, 1]
final_probs_LR_3780_2064 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_2064)[:, 1]
final_probs_LR_5580_2064 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_2064)[:, 1]

final_preds_LR_360_2064  = (final_probs_LR_360_2064  >= 0.5).astype(int)
final_preds_LR_1980_2064 = (final_probs_LR_1980_2064 >= 0.5).astype(int)
final_preds_LR_3780_2064 = (final_probs_LR_3780_2064 >= 0.5).astype(int)
final_preds_LR_5580_2064 = (final_probs_LR_5580_2064 >= 0.5).astype(int)

# --- val_3844 ---
final_probs_LR_360_3844  = meta_model_LR_360.predict_proba(test_preds_stack_360_3844)[:, 1]
final_probs_LR_1980_3844 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_3844)[:, 1]
final_probs_LR_3780_3844 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_3844)[:, 1]
final_probs_LR_5580_3844 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_3844)[:, 1]

final_preds_LR_360_3844  = (final_probs_LR_360_3844  >= 0.5).astype(int)
final_preds_LR_1980_3844 = (final_probs_LR_1980_3844 >= 0.5).astype(int)
final_preds_LR_3780_3844 = (final_probs_LR_3780_3844 >= 0.5).astype(int)
final_preds_LR_5580_3844 = (final_probs_LR_5580_3844 >= 0.5).astype(int)

# --- val_27156 ---
final_probs_LR_360_27156  = meta_model_LR_360.predict_proba(test_preds_stack_360_27156)[:, 1]
final_probs_LR_1980_27156 = meta_model_LR_1980.predict_proba(test_preds_stack_1980_27156)[:, 1]
final_probs_LR_3780_27156 = meta_model_LR_3780.predict_proba(test_preds_stack_3780_27156)[:, 1]
final_probs_LR_5580_27156 = meta_model_LR_5580.predict_proba(test_preds_stack_5580_27156)[:, 1]

final_preds_LR_360_27156  = (final_probs_LR_360_27156  >= 0.5).astype(int)
final_preds_LR_1980_27156 = (final_probs_LR_1980_27156 >= 0.5).astype(int)
final_preds_LR_3780_27156 = (final_probs_LR_3780_27156 >= 0.5).astype(int)
final_preds_LR_5580_27156 = (final_probs_LR_5580_27156 >= 0.5).astype(int)

print("✓ All ensemble predictions complete for validation sets")

In [ ]:
# --- External validation: val_1364 ---
metrics_val_360_1364  = evaluate_meta_model_with_ci(y_val_1364, final_probs_LR_360_1364,  threshold=0.5)
metrics_val_1980_1364 = evaluate_meta_model_with_ci(y_val_1364, final_probs_LR_1980_1364, threshold=0.5)
metrics_val_3780_1364 = evaluate_meta_model_with_ci(y_val_1364, final_probs_LR_3780_1364, threshold=0.5)
metrics_val_5580_1364 = evaluate_meta_model_with_ci(y_val_1364, final_probs_LR_5580_1364, threshold=0.5)

# --- External validation: val_2064 ---
metrics_val_360_2064  = evaluate_meta_model_with_ci(y_val_2064, final_probs_LR_360_2064,  threshold=0.5)
metrics_val_1980_2064 = evaluate_meta_model_with_ci(y_val_2064, final_probs_LR_1980_2064, threshold=0.5)
metrics_val_3780_2064 = evaluate_meta_model_with_ci(y_val_2064, final_probs_LR_3780_2064, threshold=0.5)
metrics_val_5580_2064 = evaluate_meta_model_with_ci(y_val_2064, final_probs_LR_5580_2064, threshold=0.5)

# --- External validation: val_3844 ---
metrics_val_360_3844  = evaluate_meta_model_with_ci(y_val_3844, final_probs_LR_360_3844,  threshold=0.5)
metrics_val_1980_3844 = evaluate_meta_model_with_ci(y_val_3844, final_probs_LR_1980_3844, threshold=0.5)
metrics_val_3780_3844 = evaluate_meta_model_with_ci(y_val_3844, final_probs_LR_3780_3844, threshold=0.5)
metrics_val_5580_3844 = evaluate_meta_model_with_ci(y_val_3844, final_probs_LR_5580_3844, threshold=0.5)

# --- External validation: val_27156 ---
metrics_val_360_27156  = evaluate_meta_model_with_ci(y_val_27156, final_probs_LR_360_27156,  threshold=0.5)
metrics_val_1980_27156 = evaluate_meta_model_with_ci(y_val_27156, final_probs_LR_1980_27156, threshold=0.5)
metrics_val_3780_27156 = evaluate_meta_model_with_ci(y_val_27156, final_probs_LR_3780_27156, threshold=0.5)
metrics_val_5580_27156 = evaluate_meta_model_with_ci(y_val_27156, final_probs_LR_5580_27156, threshold=0.5)